In [2]:
import tensorflow_gnn as tfgnn
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
import os
import tempfile
import tensorflow as tf
import sympy as sympy
import pandas as pd
import random as random
import math as math
import itertools

In [3]:
from tensorflow_gnn.models import gcn
from tensorflow_gnn.models import graph_sage
from google.protobuf import text_format as text_format
import tensorflow_gnn.proto.graph_schema_pb2 as schema_pb2
from Determine_Bonds_Get_Input import initialize_dataset
from Determine_Bonds_Get_Input import take_data_batch
from Determine_Bonds_Get_Input import convert_floats_to_records
from Determine_Bonds_Get_Input import build_graph_tensor
from Determine_Bonds_Get_Input import molecular_graph
from sympy.utilities.iterables import multiset_permutations

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 0.244
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 0
        value: 3.75
        value: 8
        value: 0.097
        value: 0.055
        value: 498
        value: 4.62
        value: 2.91
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 3.15
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 2.5
        value: 2.99
        value: 57
        value: 0.068
        value: -0.19
        value: 1.18
        value: 10.98
        value: 2.8
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 0.88
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 3.1
        value: 3.5
        value: 8
        value: 590
        value: 0.00024
        value: 0.0543
        

In [4]:
def construct_molecular_graph_schema(proto_file, proto_message):
    graph_schema = """
    # proto-file: proto_file
    # proto-message: proto_message
    node_sets {
      key: "nuclei"
      value {
        features {
          key: "z"
          value {
            description: "the atomic number of this atom"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
         }
       } 

        features {
          key: "r"
          value {
            description: "the radial seperation of the nucleus from a common origin"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
        features{
          key: "ACSF"
          value {
            description: "atom-centered symmetry function describing environment near nucleus"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }

        features{
          key: "d1"
          value {
            description: "first unoptimized SCF contraction coefficient"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
        features{
          key: "d2"
          value {
            description: "second unoptimized SCF contraction coefficient"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
        features{
          key: "d3"
          value {
            description: "thrid unoptimized SCF contraction coefficient"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }

        features{
          key: "exp1"
          value {
            description: "first Gaussian exponential term"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
          
          
        features{
          key: "exp2"
          value {
            description: "second Gaussian exponential term"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
          
        features{
          key: "exp3"
          value {
            description: "third Gaussian exponential term"
            dtype: DT_FLOAT
            shape { dim { size: -1 } }
        }
      }
    }
  }

    """
    graph_schema = graph_schema.replace("proto_file", proto_file)
    graph_schema = graph_schema.replace("proto_message", proto_message)
    return graph_schema
    

# These are test vals for n_nuclei and n_bonding pairs before the dataset is loaded in and initialized
proto_file = "//third_party/py/tensorflow_gnn/proto/graph_schema.proto"
proto_message = " tensorflow_gnn.GraphSchema"

graph_schema = construct_molecular_graph_schema(proto_file, proto_message)
graph_schema = text_format.Merge(graph_schema, schema_pb2.GraphSchema())
graph_tensor_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)
print(graph_tensor_spec)

GraphTensorSpec({'context': ContextSpec({'features': {}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, tf.int32, None), 'node_sets': {'nuclei': NodeSetSpec({'features': {'d1': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'ACSF': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'exp3': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'z': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'r': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'exp2': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'd2': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'exp1': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32), 'd3': RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, tf.int32,

In [5]:
# build initial-state graph tensor from tensor spec and graph schema
record_filepath = "test_data.tfrecord"
csv_filepath = "/users/haydenprescott/documents/test.csv"
record_filename = "test_data.tfrecord" 
input_data_batch = tf.data.TFRecordDataset(filenames = [record_filepath])

def generate_initial_graph_tensor(csv_filepath, record_filepath, record_filename):
    initial_state, variable_state = build_graph_tensor(csv_filepath, record_filepath, record_filename)
    return initial_state

def generate_variable_state_tensor(csv_filepath, record_filepath, record_filename):
    initial_state, variable_state = build_graph_tensor(csv_filepath, record_filepath, record_filename)
    return variable_state

In [6]:
initial_state_tensor = generate_initial_graph_tensor(csv_filepath, record_filepath, record_filename)
variable_state_tensor = generate_variable_state_tensor(csv_filepath, record_filepath, record_filename)
bonding_states = {"s":0, "p":(1,2), "d":(3,7), "f":(8,)}
delocalization_placeholder = sympy.symbols("None", real = True)
aromatic_rings = np.array([[delocalization_placeholder]])
aromatic_cages = np.array([[delocalization_placeholder]])
print(initial_state_tensor)
print("\n")
print(variable_state_tensor)

[[b'r' b'ACSF' b'z' b'd1' b'd2' b'd3' b'exp1' b'exp2' b'exp3']
 [b'0.0' b'3.75' b'8.0' b'0.097' b'0.055' b'498.0' b'4.62' b'2.91'
  b'0.244']
 [b'2.5' b'2.99' b'57.0' b'0.068' b'-0.19' b'1.18' b'10.98' b'2.8'
  b'3.15']
 [b'3.1' b'3.5' b'8.0' b'590.0' b'0.00024' b'0.0543' b'3.88' b'2.98'
  b'0.88']
 [b'5.4' b'5.0' b'8.0' b'0.0071' b'4.1' b'0.00044' b'2.85' b'3.25' b'0.7']
 [b'1.9' b'3.0' b'57.0' b'-0.0048' b'0.022' b'-0.095' b'2.56' b'9.58'
  b'2.98']
 [b'2.7' b'3.38' b'57.0' b'-0.00011' b'0.0091' b'0.228' b'3.19' b'7.43'
  b'0.97']
 [b'4.7' b'5.2' b'8.0' b'45.0' b'-0.088' b'0.00512' b'4.73' b'2.9'
  b'0.66']]


[[b'0.0' b'3.75' b'8.0' b'0.097' b'0.055' b'498.0' b'4.62' b'2.91'
  b'0.244']
 [b'2.5' b'2.99' b'57.0' b'0.068' b'-0.19' b'1.18' b'10.98' b'2.8'
  b'3.15']
 [b'3.1' b'3.5' b'8.0' b'590.0' b'0.00024' b'0.0543' b'3.88' b'2.98'
  b'0.88']
 [b'5.4' b'5.0' b'8.0' b'0.0071' b'4.1' b'0.00044' b'2.85' b'3.25' b'0.7']
 [b'1.9' b'3.0' b'57.0' b'-0.0048' b'0.022' b'-0.095' b'2.56' b'9.58

In [7]:
print(type(initial_state_tensor))
for i, state_graph in enumerate(initial_state_tensor.take(1)):
      print(f"Input {i}: {state_graph}")

<class 'numpy.ndarray'>
Input 0: 65
Input 1: 67
Input 2: 83
Input 3: 70


In [8]:
molecular_graph_details = molecular_graph(orbital_overlaps = np.array(["1_2", "2_1", "1_6", "6_1", "1_7", "7_1", "2_3", "3_2", "3_4", "4_3", "3_7", "7_3", "4_5", "5_4", "5_6", "6_5", "5_7", "7_5"]), n_atoms = 7, maximum_s_el = 2, maximum_p_el = 4, maximum_d_el = 0, maximum_f_el = 0)
adj_matrix = molecular_graph_details.build_adj_matrix(molecular_graph_details.n_atoms, molecular_graph_details.orbital_overlaps)
print(adj_matrix)

[[0. 1. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0.]]


In [9]:
def graph_tensor_avg_convolution(adj_matrix, n_atoms):
    D = np.array([])
    identity = np.eye(n_atoms, n_atoms)
    i = 0
    for i in np.arange(0, n_atoms):
        atom_i_row = adj_matrix[i]
        D = np.append(D, np.sum(atom_i_row))
        i += 1
    adj_normalized = adj_matrix + identity
    adj_norm_shape = np.zeros_like(adj_normalized)
    np.fill_diagonal(adj_norm_shape, D)
    D_normalized = adj_norm_shape + identity
    D_inv = np.linalg.inv(D_normalized)
    D_inv_root = scipy.linalg.sqrtm(D_inv)
    comparison = D_inv_root @ adj_normalized @ D_inv_root
    adj_normalized = np.matmul(D_inv_root, adj_normalized)
    adj_normalized = np.matmul(adj_normalized, D_inv_root)
    return adj_normalized

def compute_node_connection_degrees(adj_matrix, n_atoms):
    degree_vector = np.array([])
    i = 0
    for i in np.arange(0, n_atoms):
        atom_i_adjacency_row = adj_matrix[i]
        degree_vector = np.append(degree_vector, np.sum(atom_i_adjacency_row))
        i += 1
    return degree_vector
    

n_atoms = molecular_graph_details.n_atoms
print(graph_tensor_avg_convolution(adj_matrix, n_atoms))
print(compute_node_connection_degrees(adj_matrix, n_atoms))

[[0.25       0.28867513 0.         0.         0.         0.28867513
  0.25      ]
 [0.28867513 0.33333333 0.28867513 0.         0.         0.
  0.        ]
 [0.         0.28867513 0.25       0.28867513 0.         0.
  0.25      ]
 [0.         0.         0.28867513 0.33333333 0.28867513 0.
  0.        ]
 [0.         0.         0.         0.28867513 0.25       0.28867513
  0.25      ]
 [0.28867513 0.         0.         0.         0.28867513 0.33333333
  0.        ]
 [0.25       0.         0.25       0.         0.25       0.
  0.25      ]]
[3. 2. 3. 2. 3. 2. 3.]


In [10]:
variable_state_tensor = variable_state_tensor.astype("float")
initial_state_floats = initial_state_tensor[1:].astype("float")
initial_state_labels = initial_state_tensor[0].astype("str")
print(initial_state_labels[3])
contraction_coefficient_range = (list(initial_state_labels).index("d1"), list(initial_state_labels).index("exp1"))
exponential_range = (list(initial_state_labels).index("exp1"), len(initial_state_labels) - 1)
conv_op = graph_tensor_avg_convolution(adj_matrix, n_atoms)
print(conv_op.transpose() @ variable_state_tensor)
print(initial_state_floats, contraction_coefficient_range)
print(type(initial_state_floats), type(contraction_coefficient_range))

d1
[[ 2.67611070e+00  4.07636061e+00  3.69089653e+01  1.12938482e+01
  -6.04713318e-02  1.24907735e+02  6.42802666e+00  4.40564663e+00
   1.41534155e+00]
 [ 1.72822625e+00  3.08956139e+00  2.36188022e+01  1.70368998e+02
  -4.73869189e-02  1.44169225e+02  6.11373864e+00  2.63362988e+00
   1.37447085e+00]
 [ 4.23053356e+00  4.48151433e+00  2.27638837e+01  1.58771680e+02
   1.10677978e+00  3.55618676e-01  6.14487711e+00  3.21648456e+00
   1.49639927e+00]
 [ 3.24337567e+00  3.54305504e+00  2.14305504e+01  1.70319310e+02
   1.37308680e+00 -1.16024113e-02  2.80906787e+00  4.70909302e+00
   1.34761935e+00]
 [ 3.98826859e+00  4.46909763e+00  3.50138837e+01  1.12508178e+01
   1.16969500e+00  4.34749477e-02  3.56609781e+00  6.20305044e+00
   1.39208747e+00]
 [ 1.44848276e+00  3.07522383e+00  3.77638837e+01  2.65791807e-02
   2.52613187e-02  1.43808793e+02  3.13602080e+00  6.08221910e+00
   1.25402197e+00]
 [ 2.42500000e+00  3.86250000e+00  2.02500000e+01  1.58773050e+02
  -2.69000000e-03  1.2449

In [11]:
# define RGCN layer architecture for predictive cluster energy and structure model, set up forward and backward computation, n_layers = 11 (10 computational, 1 readout of SCF contraction coefficients) by default to ensure interdependency of electron density in all bonds on largest computed delocalized surfces
class Computational_RGCN_Layer:
    def __init__(self, input_tensor, convolution_operation, weight_paramaters, bias_paramaters, base_weights, base_biases, orbital_overlap_schemes, activation_fxn, atom_node_connections, layer_eqn_terms):
        self.input_tensor = input_tensor
        self.convolution_operation = convolution_operation
        self.weight_paramaters = weight_paramaters
        self.bias_paramaters = bias_paramaters
        self.base_weights = base_weights
        self.base_biases = base_biases
        self.orbital_overlap_schemes = orbital_overlap_schemes
        self.activation_fxn = activation_fxn
        self.atom_node_connections = atom_node_connections
        self.layer_eqn_terms = layer_eqn_terms
        
        
    def apply_convolution(self, convolution_operation, input_tensor):
        H_i = input_tensor
        H_i_conv = convolution_operation @ H_i
        H_i_conv_T = H_i_conv.transpose()
        return H_i_conv, H_i_conv_T
        
    def compute_weights_given_orbital_overlap_types(self, orbital_overlap_schemes, base_weights, weight_paramaters, layer_eqn_terms, input_tensor):
        i = 0
        ith_state_weights = np.zeros((len(input_tensor.transpose()), len(input_tensor.transpose())))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, computational_weight_paramaters will be set to an input_tensor * input_tensor matrix of ones, but will be set to the output of the update_weight_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_weights = weight_paramaters[i].transpose() @ base_weights
            ith_state_weights = ith_state_weights + overlap_type_weights
            i += 1
        b = sympy.symbols("b", real = True)
        weight_expression = sympy.concrete.summations.Sum(layer_eqn_terms[0] * layer_eqn_terms[1], (b, 1, len(orbital_overlap_schemes)))
        return ith_state_weights, weight_expression

    def compute_biases_given_orbital_overlap_types(self, orbital_overlap_schemes, base_biases, bias_paramaters, layer_eqn_terms, input_tensor):
        i = 0
        ith_state_biases = np.zeros((len(input_tensor.transpose(),)))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, computational_bias_paramaters will be set to an input_tensor * 1 vector of ones, but will be set to the output of the update_bias_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_biases = bias_paramaters[i] @ base_biases
            ith_state_biases = ith_state_biases + overlap_type_biases
            i += 1
        b = sympy.symbols("b", real = True)
        bias_expression = sympy.concrete.summations.Sum(layer_eqn_terms[2] * layer_eqn_terms[3], (b, 1, len(orbital_overlap_schemes)))
        return ith_state_biases, bias_expression

    def lin_reg_operation(self, convolution_operation, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, layer_eqn_terms, orbital_overlap_schemes, atom_node_connections):
        ith_state_weights,ith_state_weight_expression = self.compute_weights_given_orbital_overlap_types(orbital_overlap_schemes, base_weights, weight_paramaters, layer_eqn_terms, input_tensor)
        ith_state_biases,ith_state_bias_expression = self.compute_biases_given_orbital_overlap_types(orbital_overlap_schemes, base_biases, bias_paramaters, layer_eqn_terms, input_tensor)
        H_i_conv, H_i_conv_T = self.apply_convolution(convolution_operation, input_tensor)
        H_i_lin = np.array([])
        H_i_lin = H_i_lin[np.newaxis, :]
        H_i_lin_reg = (ith_state_weights @ H_i_conv_T).transpose() + ith_state_biases
        i = 0
        for i in np.arange(0, len(atom_node_connections)):
            H_i_lin = np.append(H_i_lin, H_i_lin_reg[i] * (1/atom_node_connections[i]))
            i += 1
        H_i_lin = H_i_lin.reshape(np.shape(H_i_lin_reg))  
        H_i_lin_T = H_i_lin.transpose()
        CiR = sympy.symbols("CiR", real = True)
        i = sympy.symbols("i", real = True)
        lin_reg_expression = sympy.concrete.summations.Sum((1/CiR) * (layer_eqn_terms[4] * layer_eqn_terms[7] + layer_eqn_terms[5]), (i, 1, len(atom_node_connections)))
        return H_i_lin, H_i_lin_T, lin_reg_expression

    def gelu(self, input_tensor):
        activation_output = (0.5 * input_tensor) * (1 + np.tanh(np.sqrt(2/np.pi) * (input_tensor + 0.044715*input_tensor**3)))
        return activation_output

    def forward_graph_update(self, convolution_operation, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, layer_eqn_terms, orbital_overlap_schemes, activation_fxn, atom_node_connections):
        computational_activation = activation_fxn
        H_i_conv, H_i_conv_T = self.apply_convolution(convolution_operation, input_tensor)
        ith_state_weights, ith_state_weight_expression = self.compute_weights_given_orbital_overlap_types(orbital_overlap_schemes, base_weights, weight_paramaters, layer_eqn_terms, input_tensor)
        ith_state_biases, ith_state_bias_expression = self.compute_biases_given_orbital_overlap_types(orbital_overlap_schemes, base_biases, bias_paramaters, layer_eqn_terms, input_tensor)
        H_i_lin, H_i_lin_T, computational_lin_reg = self.lin_reg_operation(convolution_operation, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, layer_eqn_terms, orbital_overlap_schemes, atom_node_connections)
        if computational_activation == "gelu":
            H_i = self.gelu(H_i_lin)
            gelu_expression = (0.5 * layer_eqn_terms[6]) * (1 + sympy.functions.elementary.hyperbolic.tanh((sympy.sqrt(2/np.pi)) * (layer_eqn_terms[6] + 0.044715*layer_eqn_terms[6]**3)))
        else:
            computational_activation = None
            pass
        return H_i, H_i_conv, H_i_conv_T, H_i_lin, H_i_lin_T, computational_lin_reg, ith_state_weights, ith_state_biases, ith_state_weight_expression, ith_state_bias_expression, gelu_expression
    

In [12]:

class Readout_RGCN_Layer:
    def __init__(self, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, orbital_overlap_schemes, activation_fxn, n_contraction_coeff, atom_node_connections, readout_layer_eqn, contraction_coefficient_range, initial_state_floats, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el):
        self.input_tensor = input_tensor
        self.weight_paramaters = weight_paramaters
        self.bias_paramaters = bias_paramaters
        self.base_weights = base_weights
        self.base_biases = base_biases
        self.orbital_overlap_schemes = orbital_overlap_schemes
        self.activation_fxn = activation_fxn
        self.n_contraction_coeff = n_contraction_coeff
        self.atom_node_connections = atom_node_connections
        self.readout_layer_eqn = readout_layer_eqn
        self.contraction_coefficient_range = contraction_coefficient_range
        self.initial_state_floats = initial_state_floats
        self.maximum_s_el = maximum_s_el
        self.maximum_p_el = maximum_p_el
        self.maximum_d_el = maximum_d_el
        self.maximum_f_el = maximum_f_el
    
    def compute_weights_given_orbital_overlap_types(self, orbital_overlap_schemes, base_weights, weight_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff):
        i = 0
        nth_state_weights = np.zeros((len(input_tensor.transpose()), n_contraction_coeff))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, readout_weight_paramaters will be set to an input_tensor * input_tensor matrix of ones, but will be set to the output of the update_weight_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_weights = weight_paramaters[i].transpose() @ base_weights
            nth_state_weights = nth_state_weights + overlap_type_weights
            i += 1
        b = sympy.symbols("b", real = True)
        weight_expression = sympy.concrete.summations.Sum(readout_layer_eqn[0] * readout_layer_eqn[1], (b, 1, len(orbital_overlap_schemes)))
        return nth_state_weights, weight_expression

    def compute_biases_given_orbital_overlap_types(self, orbital_overlap_schemes, base_biases, bias_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff):
        i = 0
        nth_state_biases = np.zeros((n_contraction_coeff,))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, readout_bias_paramaters will be set to an input_tensor * 1 vector of ones, but will be set to the output of the update_bias_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_biases = bias_paramaters[i] @ base_biases
            nth_state_biases = nth_state_biases + overlap_type_biases
            i += 1
        b = sympy.symbols("b", real = True)
        bias_expression = sympy.concrete.summations.Sum(readout_layer_eqn[2] * readout_layer_eqn[3], (b, 1, len(orbital_overlap_schemes)))
        return nth_state_biases, bias_expression

    def lin_reg_operation(self, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, readout_layer_eqn, orbital_overlap_schemes, n_contraction_coeff, atom_node_connections):
        nth_state_weights, nth_state_weight_expression = self.compute_weights_given_orbital_overlap_types(orbital_overlap_schemes, base_weights, weight_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff)
        nth_state_biases, nth_state_bias_expression = self.compute_biases_given_orbital_overlap_types(orbital_overlap_schemes, base_biases, bias_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff)
        H_n_lin = np.array([])
        H_n_lin = H_n_lin[np.newaxis, :]
        readout_input_transpose = input_tensor.transpose()
        H_n_lin_reg = (nth_state_weights.transpose() @ readout_input_transpose).transpose() + nth_state_biases
        i = 0
        for i in np.arange(0, len(atom_node_connections)):
            H_n_lin = np.append(H_n_lin, H_n_lin_reg[i] * (1/atom_node_connections[i]))
            i += 1
        H_n_lin = H_n_lin.reshape(np.shape(H_n_lin_reg))
        CiR = sympy.symbols("CiR", real = True)
        i = sympy.symbols("i", real = True)
        lin_reg_expression = sympy.concrete.summations.Sum((1/CiR) * (readout_layer_eqn[4] * readout_layer_eqn[7] + readout_layer_eqn[5]), (i, 1, len(atom_node_connections)))
        return H_n_lin, lin_reg_expression

    def Lrelu(self, input_tensor):
        activation_output = np.maximum(input_tensor, 0.01 * input_tensor)
        return activation_output

    def readout_graph_update(self, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, readout_layer_eqn, orbital_overlap_schemes, activation_fxn, n_contraction_coeff, atom_node_connections, initial_state_floats, contraction_coefficient_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el):
        readout_activation = activation_fxn
        nth_state_weights, nth_state_weight_expression = self.compute_weights_given_orbital_overlap_types(orbital_overlap_schemes, base_weights, weight_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff)
        nth_state_biases, nth_state_bias_expression = self.compute_biases_given_orbital_overlap_types(orbital_overlap_schemes, base_biases, bias_paramaters, readout_layer_eqn, input_tensor, n_contraction_coeff)
        H_n_lin, readout_lin_reg = self.lin_reg_operation(input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, readout_layer_eqn, orbital_overlap_schemes, n_contraction_coeff, atom_node_connections)
        if readout_activation == "Lrelu":
            H_n = self.Lrelu(H_n_lin)
            H_n_vals = H_n.flatten()
            i = 0
            j = 0
            for i in np.arange(0, len(initial_state_floats)):
                unoptimized_contraction_coeff = initial_state_floats[i][contraction_coefficient_range[0] : contraction_coefficient_range[1]]
                for j in np.arange(0, len(unoptimized_contraction_coeff)):
                    if unoptimized_contraction_coeff[j] == 0:
                        if contraction_coefficient_range[0] < j < contraction_coefficient_range[0] + 1:
                            H_n_vals[i][maximum_s_el - 1 : maximum_s_el + maximum_p_el] == 0
                            j += 1
                        elif contraction_coefficient_range[0] + 1 < j < contraction_coefficient_range[0] + 3:
                            H_n_vals[j][maximum_s_el + maximum_p_el - 1 : maximum_s_el + maximum_p_el + maximum_d_el] == 0
                            j += 1
                        elif contraction_coefficient_range[0] + 3 < j < contraction_coefficient_range[0] + 7:
                            H_n_vals[j][maximum_s_el + maximum_p_el + maximum_d_el - 1 : maximum_s_el + maximum_p_el + maximum_d_el + maximum_f_el] == 0
                            j += 1
                        elif j >= contraction_coefficient_range[0] + 7:
                            H_n_vals[i][maximum_s_el + maximum_p_el + maximum_d_el + maximum_f_el - 1 : len(H_n_vals[i])] == 0
                            j += 1     
                    i += 1
            H_n = H_n_vals.reshape(np.shape(H_n))
            Lrelu_expression = sympy.Piecewise((0.01 * readout_layer_eqn[6], readout_layer_eqn[6] < 0), (readout_layer_eqn[6], readout_layer_eqn[6] >= 0))
        else:
            readout_activation = None
            pass
        return H_n, H_n_lin, readout_lin_reg, nth_state_weights, nth_state_biases, nth_state_weight_expression, nth_state_bias_expression, Lrelu_expression  
       

In [13]:
# test forward compute and hidden layers
orbital_overlap_types = np.array([0,1,2,3,4])
# for the entries in the overlap types (edge connection types) vector, a 0 represents sigma overlap only (single M.O), 1 represents sigma + pi M.O's, 2 represents sigma + 2 pi M.O's, 3 represents a delta M.O, and 4 represents an M.O with delta and sigma or delta and pi characteristics within the density of states. Vector is in order from 1 to 5 atomic orbital overlaps
input_layer = Computational_RGCN_Layer(input_tensor = variable_state_tensor, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), base_weights = np.ones((len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), base_biases = np.ones((len(variable_state_tensor.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBTi", real = True), sympy.symbols("VBi", real = True), sympy.symbols("A_RBBi", real = True), sympy.symbols("VBBi", real = True), sympy.symbols("Wi", real = True), sympy.symbols("bi", real = True), sympy.symbols("H_i", real = True), sympy.symbols("H_i_conv", real = True)]))
#print(input_layer.apply_convolution(input_layer.convolution_operation, input_layer.input_tensor))                          
#print(input_layer.compute_weights_given_orbital_overlap_types(input_layer.orbital_overlap_schemes, input_layer.base_weights, input_layer.weight_paramaters, input_layer.layer_eqn_terms, input_layer.input_tensor))
#print(input_layer.compute_biases_given_orbital_overlap_types(input_layer.orbital_overlap_schemes, input_layer.base_biases, input_layer.bias_paramaters, input_layer.layer_eqn_terms, input_layer.input_tensor))
#print(input_layer.lin_reg_operation(input_layer.convolution_operation, input_layer.input_tensor, input_layer.weight_paramaters, input_layer.bias_paramaters, input_layer.base_weights, input_layer.base_biases, input_layer.layer_eqn_terms, input_layer.orbital_overlap_schemes, input_layer.atom_node_connections))
print(input_layer.forward_graph_update(input_layer.convolution_operation, input_layer.input_tensor, input_layer.weight_paramaters, input_layer.bias_paramaters, input_layer.base_weights, input_layer.base_biases, input_layer.layer_eqn_terms, input_layer.orbital_overlap_schemes, input_layer.activation_fxn, input_layer.atom_node_connections))                                   

(array([[2895.77344353, 2895.77344353, 2895.77344353, 2895.77344353,
        2895.77344353, 2895.77344353, 2895.77344353, 2895.77344353,
        2895.77344353],
       [7966.10846781, 7966.10846781, 7966.10846781, 7966.10846781,
        7966.10846781, 7966.10846781, 7966.10846781, 7966.10846781,
        7966.10846781],
       [3053.51655804, 3053.51655804, 3053.51655804, 3053.51655804,
        3053.51655804, 3053.51655804, 3053.51655804, 3053.51655804,
        3053.51655804],
       [4719.68001451, 4719.68001451, 4719.68001451, 4719.68001451,
        4719.68001451, 4719.68001451, 4719.68001451, 4719.68001451,
        4719.68001451],
       [1021.44710212, 1021.44710212, 1021.44710212, 1021.44710212,
        1021.44710212, 1021.44710212, 1021.44710212, 1021.44710212,
        1021.44710212],
       [4446.46092564, 4446.46092564, 4446.46092564, 4446.46092564,
        4446.46092564, 4446.46092564, 4446.46092564, 4446.46092564,
        4446.46092564],
       [4807.949475  , 4807.949475  , 4

In [14]:
# test readout layer
H_i, H_i_conv, H_i_conv_T, H_i_lin, H_i_lin_T, computational_lin_reg, ith_state_weights, ith_state_biases, ith_state_weight_expression, ith_state_bias_expression, gelu_expression = input_layer.forward_graph_update(input_layer.convolution_operation, input_layer.input_tensor, input_layer.weight_paramaters, input_layer.bias_paramaters, input_layer.base_weights, input_layer.base_biases, input_layer.layer_eqn_terms, input_layer.orbital_overlap_schemes, input_layer.activation_fxn, input_layer.atom_node_connections)
readout_node_features = 10
readout_layer = Readout_RGCN_Layer(input_tensor = H_i, weight_paramaters = np.ones((len(orbital_overlap_types), len(H_i.transpose()), len(H_i.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), readout_node_features, readout_node_features)), base_weights = np.ones((len(H_i.transpose()), readout_node_features)), base_biases = np.ones((readout_node_features,)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "Lrelu", n_contraction_coeff = readout_node_features, atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), readout_layer_eqn = np.array([sympy.symbols("A_RBTn", real = True), sympy.symbols("VBn", real = True), sympy.symbols("A_RBBn", real = True), sympy.symbols("VBBn", real = True), sympy.symbols("Wn_T", real = True), sympy.symbols("bn", real = True), sympy.symbols("H_n-1", real = True), sympy.symbols("H_n-1_T", real = True)]), contraction_coefficient_range = contraction_coefficient_range, initial_state_floats = initial_state_floats, maximum_s_el = molecular_graph_details.maximum_s_el, maximum_p_el = molecular_graph_details.maximum_p_el, maximum_d_el = molecular_graph_details.maximum_d_el, maximum_f_el = molecular_graph_details.maximum_f_el)
#print(readout_layer.compute_weights_given_orbital_overlap_types(readout_layer.orbital_overlap_schemes, readout_layer.base_weights, readout_layer.weight_paramaters, readout_layer.readout_layer_eqn, readout_layer.input_tensor, readout_layer.n_contraction_coeff))
#print(readout_layer.compute_biases_given_orbital_overlap_types(readout_layer.orbital_overlap_schemes, readout_layer.base_biases, readout_layer.bias_paramaters, readout_layer.readout_layer_eqn, readout_layer.input_tensor, readout_layer.n_contraction_coeff))
#print(readout_layer.lin_reg_operation(readout_layer.input_tensor, readout_layer.weight_paramaters, readout_layer.bias_paramaters, readout_layer.base_weights, readout_layer.base_biases, readout_layer.readout_layer_eqn, readout_layer.orbital_overlap_schemes, readout_layer.n_contraction_coeff, readout_layer.atom_node_connections))
print(readout_layer.readout_graph_update(readout_layer.input_tensor, readout_layer.weight_paramaters, readout_layer.bias_paramaters, readout_layer.base_weights, readout_layer.base_biases, readout_layer.readout_layer_eqn, readout_layer.orbital_overlap_schemes, readout_layer.activation_fxn, readout_layer.n_contraction_coeff, readout_layer.atom_node_connections, readout_layer.initial_state_floats, readout_layer.contraction_coefficient_range, readout_layer.maximum_s_el, readout_layer.maximum_p_el, readout_layer.maximum_d_el, readout_layer.maximum_f_el))


(array([[ 390946.08154375,  390946.08154375,  390946.08154375,
         390946.08154375,  390946.08154375,  390946.08154375,
         390946.08154375,  390946.08154375,  390946.08154375,
         390946.08154375],
       [1613161.96473213, 1613161.96473213, 1613161.96473213,
        1613161.96473213, 1613161.96473213, 1613161.96473213,
        1613161.96473213, 1613161.96473213, 1613161.96473213,
        1613161.96473213],
       [ 412241.4020022 ,  412241.4020022 ,  412241.4020022 ,
         412241.4020022 ,  412241.4020022 ,  412241.4020022 ,
         412241.4020022 ,  412241.4020022 ,  412241.4020022 ,
         412241.4020022 ],
       [ 955760.20293921,  955760.20293921,  955760.20293921,
         955760.20293921,  955760.20293921,  955760.20293921,
         955760.20293921,  955760.20293921,  955760.20293921,
         955760.20293921],
       [ 137912.0254525 ,  137912.0254525 ,  137912.0254525 ,
         137912.0254525 ,  137912.0254525 ,  137912.0254525 ,
         137912.0254525

In [15]:
# build forward computation loop, get contraction coefficient prediction for wavefunction of bonding electrons in each involved atom in cluster
hidden_state_eqns = np.array([])
weight_eqns = np.array([])
bias_eqns = np.array([])
lin_eqns = np.array([])
hidden_states = np.array([])
input_layer = Computational_RGCN_Layer(input_tensor = variable_state_tensor, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), base_weights = np.ones((len(variable_state_tensor.transpose()), len(variable_state_tensor.transpose()))), base_biases = np.ones((len(variable_state_tensor.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT1", real = True), sympy.symbols("VB1", real = True), sympy.symbols("A_RBB1", real = True), sympy.symbols("VBB1", real = True), sympy.symbols("W1", real = True), sympy.symbols("B1", real = True), sympy.symbols("H_1", real = True), sympy.symbols("H_o_conv", real = True)]))
H_1_val, H_o_conv, H_o_conv_T, H_1_lin_val, H_1_lin_T_val, H_1_lin, W1_vals, B1_vals, W1, B1, H1 = input_layer.forward_graph_update(input_layer.convolution_operation, input_layer.input_tensor, input_layer.weight_paramaters, input_layer.bias_paramaters, input_layer.base_weights, input_layer.base_biases, input_layer.layer_eqn_terms, input_layer.orbital_overlap_schemes, input_layer.activation_fxn, input_layer.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H1)
weight_eqns = np.append(weight_eqns, W1)
bias_eqns = np.append(bias_eqns, B1)
lin_eqns = np.append(lin_eqns, H_1_lin)
hidden_states = np.append(hidden_states, H_1_val)

first_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_1_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_1_val.transpose()), len(H_1_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_1_val.transpose()), len(H_1_val.transpose()))), base_weights = np.ones((len(H_1_val.transpose()), len(H_1_val.transpose()))), base_biases = np.ones((len(H_1_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT2", real = True), sympy.symbols("VB2", real = True), sympy.symbols("A_RBB2", real = True), sympy.symbols("VBB2", real = True), sympy.symbols("W2", real = True), sympy.symbols("B2", real = True), sympy.symbols("H_2", real = True), sympy.symbols("H_1_conv", real = True)]))
H_2_val, H_1_conv, H_1_conv_T, H_2_lin_val, H_2_lin_T_val, H_2_lin, W2_vals, B2_vals, W2, B2, H2 = first_hidden_encoding.forward_graph_update(first_hidden_encoding.convolution_operation, first_hidden_encoding.input_tensor, first_hidden_encoding.weight_paramaters, first_hidden_encoding.bias_paramaters, first_hidden_encoding.base_weights, first_hidden_encoding.base_biases, first_hidden_encoding.layer_eqn_terms, first_hidden_encoding.orbital_overlap_schemes, first_hidden_encoding.activation_fxn, first_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H2)
weight_eqns = np.append(weight_eqns, W2)
bias_eqns = np.append(bias_eqns, B2)
lin_eqns = np.append(lin_eqns, H_2_lin)
hidden_states = np.append(hidden_states, H_2_val)

second_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_2_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_2_val.transpose()), len(H_2_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_2_val.transpose()), len(H_2_val.transpose()))), base_weights = np.ones((len(H_2_val.transpose()), len(H_2_val.transpose()))), base_biases = np.ones((len(H_2_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT3", real = True), sympy.symbols("VB3", real = True), sympy.symbols("A_RBB3", real = True), sympy.symbols("VBB3", real = True), sympy.symbols("W3", real = True), sympy.symbols("B3", real = True), sympy.symbols("H_3", real = True), sympy.symbols("H_2_conv", real = True)]))
H_3_val, H_2_conv, H_2_conv_T, H_3_lin_val, H_3_lin_T_val, H_3_lin, W3_vals, B3_vals, W3, B3, H3 = second_hidden_encoding.forward_graph_update(second_hidden_encoding.convolution_operation, second_hidden_encoding.input_tensor, second_hidden_encoding.weight_paramaters, second_hidden_encoding.bias_paramaters, second_hidden_encoding.base_weights, second_hidden_encoding.base_biases, second_hidden_encoding.layer_eqn_terms, second_hidden_encoding.orbital_overlap_schemes, second_hidden_encoding.activation_fxn, second_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H3)
weight_eqns = np.append(weight_eqns, W3)
bias_eqns = np.append(bias_eqns, B3)
lin_eqns = np.append(lin_eqns, H_3_lin)
hidden_states = np.append(hidden_states, H_3_val)

third_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_3_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_3_val.transpose()), len(H_3_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_3_val.transpose()), len(H_3_val.transpose()))), base_weights = np.ones((len(H_3_val.transpose()), len(H_3_val.transpose()))), base_biases = np.ones((len(H_3_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT4", real = True), sympy.symbols("VB4", real = True), sympy.symbols("A_RBB4", real = True), sympy.symbols("VBB4", real = True), sympy.symbols("W4", real = True), sympy.symbols("B4", real = True), sympy.symbols("H_4", real = True), sympy.symbols("H_3_conv", real = True)]))
H_4_val, H_3_conv, H_3_conv_T, H_4_lin_val, H_4_lin_T_val, H_4_lin, W4_vals, B4_vals, W4, B4, H4 = third_hidden_encoding.forward_graph_update(third_hidden_encoding.convolution_operation, third_hidden_encoding.input_tensor, third_hidden_encoding.weight_paramaters, third_hidden_encoding.bias_paramaters, third_hidden_encoding.base_weights, third_hidden_encoding.base_biases, third_hidden_encoding.layer_eqn_terms, third_hidden_encoding.orbital_overlap_schemes, third_hidden_encoding.activation_fxn, third_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H4)
weight_eqns = np.append(weight_eqns, W4)
bias_eqns = np.append(bias_eqns, B4)
lin_eqns = np.append(lin_eqns, H_4_lin)
hidden_states = np.append(hidden_states, H_4_val)

fourth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_4_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_4_val.transpose()), len(H_4_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_4_val.transpose()), len(H_4_val.transpose()))), base_weights = np.ones((len(H_4_val.transpose()), len(H_4_val.transpose()))), base_biases = np.ones((len(H_4_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT5", real = True), sympy.symbols("VB5", real = True), sympy.symbols("A_RBB5", real = True), sympy.symbols("VBB5", real = True), sympy.symbols("W5", real = True), sympy.symbols("B5", real = True), sympy.symbols("H_5", real = True), sympy.symbols("H_4_conv", real = True)]))
H_5_val, H_4_conv, H_4_conv_T, H_5_lin_val, H_5_lin_T_val, H_5_lin, W5_vals, B5_vals, W5, B5, H5 = fourth_hidden_encoding.forward_graph_update(fourth_hidden_encoding.convolution_operation, fourth_hidden_encoding.input_tensor, fourth_hidden_encoding.weight_paramaters, fourth_hidden_encoding.bias_paramaters, fourth_hidden_encoding.base_weights, fourth_hidden_encoding.base_biases, fourth_hidden_encoding.layer_eqn_terms, fourth_hidden_encoding.orbital_overlap_schemes, fourth_hidden_encoding.activation_fxn, fourth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H5)
weight_eqns = np.append(weight_eqns, W5)
bias_eqns = np.append(bias_eqns, B5)
lin_eqns = np.append(lin_eqns, H_5_lin)
hidden_states = np.append(hidden_states, H_5_val)

fifth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_5_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_5_val.transpose()), len(H_5_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_5_val.transpose()), len(H_5_val.transpose()))), base_weights = np.ones((len(H_5_val.transpose()), len(H_5_val.transpose()))), base_biases = np.ones((len(H_5_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT6", real = True), sympy.symbols("VB6", real = True), sympy.symbols("A_RBB6", real = True), sympy.symbols("VBB6", real = True), sympy.symbols("W6", real = True), sympy.symbols("B6", real = True), sympy.symbols("H_6", real = True), sympy.symbols("H_5_conv", real = True)]))
H_6_val, H_5_conv, H_5_conv_T, H_6_lin_val, H_6_lin_T_val, H_6_lin, W6_vals, B6_vals, W6, B6, H6 = fifth_hidden_encoding.forward_graph_update(fifth_hidden_encoding.convolution_operation, fifth_hidden_encoding.input_tensor, fifth_hidden_encoding.weight_paramaters, fifth_hidden_encoding.bias_paramaters, fifth_hidden_encoding.base_weights, fifth_hidden_encoding.base_biases, fifth_hidden_encoding.layer_eqn_terms, fifth_hidden_encoding.orbital_overlap_schemes, fifth_hidden_encoding.activation_fxn, fifth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H6)
weight_eqns = np.append(weight_eqns, W6)
bias_eqns = np.append(bias_eqns, B6)
lin_eqns = np.append(lin_eqns, H_6_lin)
hidden_states = np.append(hidden_states, H_6_val)

sixth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_6_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_6_val.transpose()), len(H_6_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_6_val.transpose()), len(H_6_val.transpose()))), base_weights = np.ones((len(H_6_val.transpose()), len(H_6_val.transpose()))), base_biases = np.ones((len(H_6_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT7", real = True), sympy.symbols("VB7", real = True), sympy.symbols("A_RBB7", real = True), sympy.symbols("VBB7", real = True), sympy.symbols("W7", real = True), sympy.symbols("B7", real = True), sympy.symbols("H_7", real = True), sympy.symbols("H_6_conv", real = True)]))
H_7_val, H_6_conv, H_6_conv_T, H_7_lin_val, H_7_lin_T_val, H_7_lin, W7_vals, B7_vals, W7, B7, H7 = sixth_hidden_encoding.forward_graph_update(sixth_hidden_encoding.convolution_operation, sixth_hidden_encoding.input_tensor, sixth_hidden_encoding.weight_paramaters, sixth_hidden_encoding.bias_paramaters, sixth_hidden_encoding.base_weights, sixth_hidden_encoding.base_biases, sixth_hidden_encoding.layer_eqn_terms, sixth_hidden_encoding.orbital_overlap_schemes, sixth_hidden_encoding.activation_fxn, sixth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H7)
weight_eqns = np.append(weight_eqns, W7)
bias_eqns = np.append(bias_eqns, B7)
lin_eqns = np.append(lin_eqns, H_7_lin)
hidden_states = np.append(hidden_states, H_7_val)

seventh_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_7_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_7_val.transpose()), len(H_7_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_7_val.transpose()), len(H_7_val.transpose()))), base_weights = np.ones((len(H_7_val.transpose()), len(H_7_val.transpose()))), base_biases = np.ones((len(H_7_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT8", real = True), sympy.symbols("VB8", real = True), sympy.symbols("A_RBB8", real = True), sympy.symbols("VBB8", real = True), sympy.symbols("W8", real = True), sympy.symbols("B8", real = True), sympy.symbols("H_8", real = True), sympy.symbols("H_7_conv", real = True)]))
H_8_val, H_7_conv, H_7_conv_T, H_8_lin_val, H_8_lin_T_val, H_8_lin, W8_vals, B8_vals, W8, B8, H8 = seventh_hidden_encoding.forward_graph_update(seventh_hidden_encoding.convolution_operation, seventh_hidden_encoding.input_tensor, seventh_hidden_encoding.weight_paramaters, seventh_hidden_encoding.bias_paramaters, seventh_hidden_encoding.base_weights, seventh_hidden_encoding.base_biases, seventh_hidden_encoding.layer_eqn_terms, seventh_hidden_encoding.orbital_overlap_schemes, seventh_hidden_encoding.activation_fxn, seventh_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H8)
weight_eqns = np.append(weight_eqns, W8)
bias_eqns = np.append(bias_eqns, B8)
lin_eqns = np.append(lin_eqns, H_8_lin)
hidden_states = np.append(hidden_states, H_8_val)

eighth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_8_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_8_val.transpose()), len(H_8_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_8_val.transpose()), len(H_8_val.transpose()))), base_weights = np.ones((len(H_8_val.transpose()), len(H_8_val.transpose()))), base_biases = np.ones((len(H_8_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT9", real = True), sympy.symbols("VB9", real = True), sympy.symbols("A_RBB9", real = True), sympy.symbols("VBB9", real = True), sympy.symbols("W9", real = True), sympy.symbols("B9", real = True), sympy.symbols("H_9", real = True), sympy.symbols("H_8_conv", real = True)]))
H_9_val, H_8_conv, H_8_conv_T, H_9_lin_val, H_9_lin_T_val, H_9_lin, W9_vals, B9_vals, W9, B9, H9 = eighth_hidden_encoding.forward_graph_update(eighth_hidden_encoding.convolution_operation, eighth_hidden_encoding.input_tensor, eighth_hidden_encoding.weight_paramaters, eighth_hidden_encoding.bias_paramaters, eighth_hidden_encoding.base_weights, eighth_hidden_encoding.base_biases, eighth_hidden_encoding.layer_eqn_terms, eighth_hidden_encoding.orbital_overlap_schemes, eighth_hidden_encoding.activation_fxn, eighth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H9)
weight_eqns = np.append(weight_eqns, W9)
bias_eqns = np.append(bias_eqns, B9)
lin_eqns = np.append(lin_eqns, H_9_lin)
hidden_states = np.append(hidden_states, H_9_val)

ninth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_9_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_9_val.transpose()), len(H_9_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_9_val.transpose()), len(H_9_val.transpose()))), base_weights = np.ones((len(H_9_val.transpose()), len(H_9_val.transpose()))), base_biases = np.ones((len(H_9_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT10", real = True), sympy.symbols("VB10", real = True), sympy.symbols("A_RBB9", real = True), sympy.symbols("VBB9", real = True), sympy.symbols("W10", real = True), sympy.symbols("B10", real = True), sympy.symbols("H_10", real = True), sympy.symbols("H_9_conv", real = True)]))
H_10_val, H_9_conv, H_9_conv_T, H_10_lin_val, H_10_lin_T_val, H_10_lin, W10_vals, B10_vals, W10, B10, H10 = ninth_hidden_encoding.forward_graph_update(ninth_hidden_encoding.convolution_operation, ninth_hidden_encoding.input_tensor, ninth_hidden_encoding.weight_paramaters, ninth_hidden_encoding.bias_paramaters, ninth_hidden_encoding.base_weights, ninth_hidden_encoding.base_biases, ninth_hidden_encoding.layer_eqn_terms, ninth_hidden_encoding.orbital_overlap_schemes, ninth_hidden_encoding.activation_fxn, ninth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H10)
weight_eqns = np.append(weight_eqns, W10)
bias_eqns = np.append(bias_eqns, B10)
lin_eqns = np.append(lin_eqns, H_10_lin)
hidden_states = np.append(hidden_states, H_10_val)

tenth_hidden_encoding = Computational_RGCN_Layer(input_tensor = H_10_val, convolution_operation = graph_tensor_avg_convolution(adj_matrix, n_atoms), weight_paramaters = np.ones((len(orbital_overlap_types), len(H_10_val.transpose()), len(H_10_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), len(H_10_val.transpose()), len(H_10_val.transpose()))), base_weights = np.ones((len(H_10_val.transpose()), len(H_10_val.transpose()))), base_biases = np.ones((len(H_10_val.transpose()),)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "gelu", atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), layer_eqn_terms = np.array([sympy.symbols("A_RBT11", real = True), sympy.symbols("VB11", real = True), sympy.symbols("A_RBB10", real = True), sympy.symbols("VBB10", real = True), sympy.symbols("W11", real = True), sympy.symbols("B11", real = True), sympy.symbols("H_11", real = True), sympy.symbols("H_10_conv", real = True)]))
H_11_val, H_10_conv, H_10_conv_T, H_11_lin_val, H_11_lin_T_val, H_11_lin, W11_vals, B11_vals, W11, B11, H11 = tenth_hidden_encoding.forward_graph_update(tenth_hidden_encoding.convolution_operation, tenth_hidden_encoding.input_tensor, tenth_hidden_encoding.weight_paramaters, tenth_hidden_encoding.bias_paramaters, tenth_hidden_encoding.base_weights, tenth_hidden_encoding.base_biases, tenth_hidden_encoding.layer_eqn_terms, tenth_hidden_encoding.orbital_overlap_schemes, tenth_hidden_encoding.activation_fxn, tenth_hidden_encoding.atom_node_connections)
hidden_state_eqns = np.append(hidden_state_eqns, H11)
weight_eqns = np.append(weight_eqns, W11)
bias_eqns = np.append(bias_eqns, B11)
lin_eqns = np.append(lin_eqns, H_11_lin)
hidden_states = np.append(hidden_states, H_11_val)

readout_layer = Readout_RGCN_Layer(input_tensor = H_11_val, weight_paramaters = np.ones((len(orbital_overlap_types), len(H_11_val.transpose()), len(H_11_val.transpose()))), bias_paramaters = np.ones((len(orbital_overlap_types), readout_node_features, readout_node_features)), base_weights = np.ones((len(H_11_val.transpose()), readout_node_features)), base_biases = np.ones((readout_node_features,)), orbital_overlap_schemes = orbital_overlap_types, activation_fxn = "Lrelu", n_contraction_coeff = readout_node_features, atom_node_connections = compute_node_connection_degrees(adj_matrix, n_atoms), readout_layer_eqn = np.array([sympy.symbols("A_RBTf", real = True), sympy.symbols("VBf", real = True), sympy.symbols("A_RBBf", real = True), sympy.symbols("VBBf", real = True), sympy.symbols("Wf_T", real = True), sympy.symbols("bf", real = True), sympy.symbols("H_f-1", real = True), sympy.symbols("H_f-1_T", real = True)]), contraction_coefficient_range = contraction_coefficient_range, initial_state_floats = initial_state_floats, maximum_s_el = molecular_graph_details.maximum_s_el, maximum_p_el = molecular_graph_details.maximum_p_el, maximum_d_el = molecular_graph_details.maximum_d_el, maximum_f_el = molecular_graph_details.maximum_f_el)
H_f_val, H_f_lin_val, H_f_lin, Wf_val, Bf_val, Wf, Bf, H_f = readout_layer.readout_graph_update(readout_layer.input_tensor, readout_layer.weight_paramaters, readout_layer.bias_paramaters, readout_layer.base_weights, readout_layer.base_biases, readout_layer.readout_layer_eqn, readout_layer.orbital_overlap_schemes, readout_layer.activation_fxn, readout_layer.n_contraction_coeff, readout_layer.atom_node_connections, readout_layer.initial_state_floats, readout_layer.contraction_coefficient_range, readout_layer.maximum_s_el, readout_layer.maximum_p_el, readout_layer.maximum_d_el, readout_layer.maximum_f_el)
hidden_state_eqns = np.append(hidden_state_eqns, H_f)
weight_eqns = np.append(weight_eqns, Wf)
bias_eqns = np.append(bias_eqns, Bf)
lin_eqns = np.append(lin_eqns, H_f_lin)


#print(H_f_val, hidden_state_eqns, weight_eqns, bias_eqns, lin_eqns)
print(hidden_states)
print(np.shape(H_f_val))



[2.89577344e+03 2.89577344e+03 2.89577344e+03 2.89577344e+03
 2.89577344e+03 2.89577344e+03 2.89577344e+03 2.89577344e+03
 2.89577344e+03 7.96610847e+03 7.96610847e+03 7.96610847e+03
 7.96610847e+03 7.96610847e+03 7.96610847e+03 7.96610847e+03
 7.96610847e+03 7.96610847e+03 3.05351656e+03 3.05351656e+03
 3.05351656e+03 3.05351656e+03 3.05351656e+03 3.05351656e+03
 3.05351656e+03 3.05351656e+03 3.05351656e+03 4.71968001e+03
 4.71968001e+03 4.71968001e+03 4.71968001e+03 4.71968001e+03
 4.71968001e+03 4.71968001e+03 4.71968001e+03 4.71968001e+03
 1.02144710e+03 1.02144710e+03 1.02144710e+03 1.02144710e+03
 1.02144710e+03 1.02144710e+03 1.02144710e+03 1.02144710e+03
 1.02144710e+03 4.44646093e+03 4.44646093e+03 4.44646093e+03
 4.44646093e+03 4.44646093e+03 4.44646093e+03 4.44646093e+03
 4.44646093e+03 4.44646093e+03 4.80794947e+03 4.80794947e+03
 4.80794947e+03 4.80794947e+03 4.80794947e+03 4.80794947e+03
 4.80794947e+03 4.80794947e+03 4.80794947e+03 7.43747667e+05
 7.43747667e+05 7.437476

In [24]:
# Set up gradient descent algorithm for backpropagation, Lrelu + Gelu activation optimization on DFT-calculated cluster energy dataset
class System_Energy_Optimizer:
    def __init__(self, LR_o, K, t, input_tensor, C_i_C_n, H_o_H_f, hidden_state_eqns, W_o_W_f, B_o_B_f, lin_regs, E_xc, n_bonding_electrons, n_atoms, unit_charge, atomic_orbital_states, R_inner, delta_o_cut, maximum_density_bound, gradient_descent_schedule, exponential_range, nuclear_separation_matrix, bond_separation_matrix, pos_matrix, degree_vector, atomic_positions, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, aromatic_rings, aromatic_cages, orbital_tracker, KS_functional_tracker):
        self.LR_o = LR_o
        self.K = K
        self.t = t
        self.input_tensor = input_tensor
        self.C_i_C_n = C_i_C_n
        self.H_o_H_f = H_o_H_f
        self.hidden_state_eqns = hidden_state_eqns
        self.W_o_W_f = W_o_W_f
        self.B_o_B_f = B_o_B_f
        self.lin_regs = lin_regs
        self.E_xc = E_xc
        self.n_bonding_electrons = n_bonding_electrons
        self.n_atoms = n_atoms
        self.unit_charge = unit_charge
        self.atomic_orbital_states = atomic_orbital_states
        self.R_inner = R_inner
        self.delta_o_cut = delta_o_cut
        self.maximum_density_bound = maximum_density_bound
        self.gradient_descent_schedule = gradient_descent_schedule
        self.exponential_range = exponential_range
        self.nuclear_separation_matrix = nuclear_separation_matrix
        self.bond_separation_matrix = bond_separation_matrix
        self.pos_matrix = pos_matrix
        self.degree_vector = degree_vector
        self.atomic_positions = atomic_positions
        self.maximum_s_el = maximum_s_el
        self.maximum_p_el = maximum_p_el
        self.maximum_d_el = maximum_d_el
        self.maximum_f_el = maximum_f_el
        self.aromatic_rings = aromatic_rings
        self.aromatic_cages = aromatic_cages
        self.orbital_tracker = orbital_tracker
        self.KS_functional_tracker = KS_functional_tracker

    
    def construct_grad_descent_schedule(self, LR_o, t, K, gradient_descent_schedule):
        # implement RMS normalization for stochastic gradient descent - calculate gradients as cumulative moving average of sqaures, normalize by square root of mean-square gradient, and update weights in direction of gloabl min in accordance with sign of previous gradient with respect to the previous weight 
        # paper link: https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a
        if gradient_descent_schedule == "MBGD":
            LR = LR_o * np.exp(-K * t)
        return LR
   
    def construct_single_electron_wavefunctions(self, input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el):
        s_MO_set = sympy.Array([])
        p_MO_set = sympy.Array([])
        d_MO_set = sympy.Array([])
        f_MO_set = sympy.Array([])
        s_ref_set = sympy.Array([])
        p_ref_set = sympy.Array([])
        d_ref_set = sympy.Array([])
        f_ref_set = sympy.Array([])
        s_exp_set = sympy.Array([])
        p_exp_set = sympy.Array([])
        d_exp_set = sympy.Array([])
        f_exp_set = sympy.Array([])
        exp_set = sympy.Array([])
        one_electron_MOs = np.array([])
        gaussian_terms = []
        coefficient_sequence = []
        exp_set = []
        r = sympy.symbols("r", real = True)
        j = sympy.symbols("j", real = True)
        s_exponentials = 1
        p_exponentials = atomic_orbital_states["p"][1] - atomic_orbital_states["s"]
        d_exponentials = atomic_orbital_states["d"][1] - atomic_orbital_states["p"][1]
        f_exponentials = atomic_orbital_states["f"][0]
        i = 0
        for i in range(0, n_atoms):
            if len(C_i_C_n.transpose()) > s_exponentials - 1:
                exp_start = exponential_range[s_exponentials - 1]
                s_coefficient_range = C_i_C_n[i][0:maximum_s_el * s_exponentials]
                j = 0
                for j in range(0, len(s_coefficient_range)):
                    current_contraction_coeff = s_coefficient_range[j]
                    if current_contraction_coeff != 0:
                        current_s_MO = current_contraction_coeff * sympy.exp(-input_tensor[i][exp_start] * r)
                        s_MO_set = np.append(s_MO_set, np.array([current_s_MO, i + 1]))
                        s_ref_set = np.append(s_ref_set, current_s_MO)
                        s_exp_set = np.append(s_exp_set, sympy.exp(-input_tensor[i][exp_start] * r))
                        gaussian_terms.append(current_s_MO)
                        coefficient_sequence.append(current_contraction_coeff)
                        j += s_exponentials
            else:
                pass
            exp_range_vals = np.arange(exponential_range[0], exponential_range[1] + 1)
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el:
                p_coefficient_range = C_i_C_n[i][maximum_s_el * s_exponentials : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials]
                j = 0
                p_MO_contributions = 0
                while j <= len(p_coefficient_range):
                    current_contraction_coeffs = p_coefficient_range[j: j + p_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["p"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            p_exp_set = np.append(p_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["p"][0]] * r))
                            p_MO_contributions = p_MO_contributions + current_gaussian_term
                            k += 1
                    j += p_exponentials
                    if j <= len(p_coefficient_range):
                        p_MO_set = np.append(p_MO_set, np.array([p_MO_contributions, i + 1]))  
                        p_ref_set = np.append(p_ref_set, p_MO_contributions)
            else:
                pass
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el + p_exponentials * maximum_p_el:
                d_coefficient_range = C_i_C_n[i][maximum_s_el * s_exponentials + maximum_p_el * p_exponentials : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials + maximum_d_el + d_exponentials]
                j = 0
                d_MO_contributions = 0
                while j <= len(d_coefficient_range):
                    current_contraction_coeffs = d_coefficient_range[j: j + d_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["d"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            d_exp_set = np.append(d_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["d"][0]] * r))
                            d_MO_contributions = d_MO_contributions + current_gaussian_term
                            k += 1
                    j += d_exponentials
                    if j <= len(d_coefficient_range):
                        d_MO_set = np.append(d_MO_set, np.array([d_MO_contributions, i + 1])) 
                        d_ref_set = np.append(d_ref_set, d_MO_contributions)
            else:
                pass
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el + p_exponentials * maximum_p_el + d_exponentials * maximum_d_el:
                f_coefficient_range = C_i_C_n[i][s_exponentials * maximum_s_el + p_exponentials * maximum_p_el + d_exponentials * maximum_d_el : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials + maximum_d_el * d_exponentials + maximum_f_el * f_exponentials]
                j = 0
                f_MO_contributions = 0
                while j <= len(f_coefficient_range):
                    current_contraction_coeffs = d_coefficient_range[j: j + f_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["f"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            f_exp_set = np.append(f_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["f"][0]] * r))
                            f_MO_contributions = f_MO_contributions + current_gaussian_term
                            k += 1
                    j += f_exponentials
                    if j <= len(f_coefficient_range):
                        f_MO_set = np.append(f_MO_set, np.array([f_MO_contributions, i + 1])) 
                        f_ref_set = np.append(f_ref_set, f_MO_contributions)
            else:
                pass
            i += 1
        Ci = sympy.symbols("Ci", real = True)
        one_electron_MOs = np.append(one_electron_MOs, s_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, p_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, d_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, f_MO_set)
        exp_set = np.append(exp_set, s_exp_set)
        exp_set = np.append(exp_set, p_exp_set)
        exp_set = np.append(exp_set, d_exp_set)
        exp_set = np.append(exp_set, f_exp_set)
        one_electron_MO_eqns = np.array([])
        k = 0
        for k in range(0, len(s_ref_set) * s_exponentials):
            current_s_MO_eqn = Ci * exp_set[k]
            one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_s_MO_eqn)
            k += 1
        p_gaussian_start_index = (len(s_ref_set) * s_exponentials) - 1
        p_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials) - 1
        p_gaussian_count = p_gaussian_end_index - p_gaussian_start_index
        k = 0
        for k in range(0, int(p_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials])
            current_ref_fxns = list(gaussian_terms[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials]
            current_exponentials = exp_set[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials* k + p_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
        d_gaussian_start_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials) - 1
        d_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials) - 1
        d_gaussian_count = d_gaussian_end_index - d_gaussian_start_index
        k = 0
        for k in range(0, int(d_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials])
            current_ref_fxns = list(gaussian_terms[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials]
            current_exponentials = exp_set[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials* k + d_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = random.choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
        f_gaussian_start_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials) - 1
        f_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials + len(f_ref_set) * f_exponentials) - 1
        f_gaussian_count = f_gaussian_end_index - f_gaussian_start_index
        k = 0
        for k in range(0, int(f_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials])
            current_ref_fxns = list(gaussian_terms[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials]
            current_exponentials = exp_set[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials* k + f_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = random.choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
                    
        return one_electron_MOs, one_electron_MO_eqns, r

    def compute_effective_density_cutoffs(self, input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, maximum_density_bound, bond_separation_matrix, pos_matrix, nuclear_separation_matrix):
        single_el_wavefunctions, single_el_wavefunction_eqs, r = self.construct_single_electron_wavefunctions(energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.n_atoms, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el)
        normalized_dist_fxns = np.array([])
        maximum_el_density_radii = np.array([])
        i = 0
        while i < len(single_el_wavefunctions):
            norm_constant = sympy.symbols("A", real = True)
            normalized_wavefunc = norm_constant * single_el_wavefunctions[i]
            total_probability_density = sympy.integrals.integrate(normalized_wavefunc, (r, R_inner, maximum_density_bound))
            norm_constant = sympy.solve(total_probability_density - 1, norm_constant)
            normalized_wavefunc = norm_constant[0] * single_el_wavefunctions[i]
            normalized_dist_fxns = np.append(normalized_dist_fxns, normalized_wavefunc)
            enclosed_probability_density = sympy.integrals.integrate(normalized_wavefunc, r)
            inner_cutoff_single_electron_density = enclosed_probability_density.replace(r, R_inner)
            ith_electron_density_range = ((enclosed_probability_density - inner_cutoff_single_electron_density) - 0.98)**2
            ith_density_range_fxn = sympy.lambdify(r, ith_electron_density_range)
            min_square_enclosed_density = scipy.optimize.minimize_scalar(ith_density_range_fxn)
            ith_maximum_density_radius = min_square_enclosed_density.x
            maximum_el_density_radii = np.append(maximum_el_density_radii, ith_maximum_density_radius)
            maximum_el_density_radii = np.append(maximum_el_density_radii, single_el_wavefunctions[i + 1])
            i += 2
        j = 0
        k = 0
        domain_overlap_matrix = np.zeros((int((len(maximum_el_density_radii)/2)), int((len(maximum_el_density_radii)/2))))
        inner_overlap_cutoff_matrix = np.zeros((int((len(maximum_el_density_radii)/2)), int((len(maximum_el_density_radii)/2))))
        outer_overlap_cutoff_matrix = np.zeros((int((len(maximum_el_density_radii)/2)), int((len(maximum_el_density_radii)/2))))
        while j < len(maximum_el_density_radii):
            jth_domain_center = pos_matrix[single_el_wavefunctions[j+1]-1]
            while k < len(maximum_el_density_radii):
                kth_domain_center = pos_matrix[single_el_wavefunctions[k+1]-1]
                if k == j:
                    overlap_state = 0
                    domain_overlap_matrix[j][k] = overlap_state
                    print(overlap_state)
                elif single_el_wavefunctions[k+1] == single_el_wavefunctions[j+1]:
                    overlap_state = 1
                    domain_overlap_matrix[int(j/2)][int(k/2)] = overlap_state
                    inner_overlap_cutoff_matrix[int(j/2)][int(k/2)] = 0
                    if maximum_el_density_radii[j] == maximum_el_density_radii[k]:
                        outer_overlap_cutoff_matrix[int(j/2)][int(k/2)] = maximum_el_density_radii[j]
                    else:
                        outer_overlap_cutoff_matrix[int(j/2)][int(k/2)] = min(np.array([maximum_el_density_radii[j], maximum_el_density_radii[k]]))
                else:
                    x_pos = sympy.symbols("x", real = True)
                    y_pos = sympy.symbols("y", real = True)
                    z_pos = sympy.symbols("z", real = True)
                    k_max_density_enclosure_sphere = sympy.sqrt((x_pos -  pos_matrix[single_el_wavefunctions[k+1]-1][0])**2 + (y_pos -  pos_matrix[single_el_wavefunctions[k+1]-1][1])**2 + (z_pos -  pos_matrix[single_el_wavefunctions[k+1]-1][2])**2) - maximum_el_density_radii[k]
                    jth_nucleus_separation_function = sympy.sqrt((x_pos - pos_matrix[single_el_wavefunctions[j+1]-1][0])**2 + (y_pos - pos_matrix[single_el_wavefunctions[j+1]-1][1])**2 + (z_pos - pos_matrix[single_el_wavefunctions[j+1]-1][2])**2)   
                    constraint_surface = k_max_density_enclosure_sphere + maximum_el_density_radii[k]
                    constraint_gradient = np.array([sympy.diff(constraint_surface, x_pos), sympy.diff(constraint_surface, y_pos), sympy.diff(constraint_surface, z_pos)])
                    nucleus_separation_gradient = np.array([sympy.diff(jth_nucleus_separation_function, x_pos), sympy.diff(jth_nucleus_separation_function, y_pos), sympy.diff(jth_nucleus_separation_function, z_pos)])
                    lagrangian = sympy.symbols("lambda", real = True)
                    min_point_and_lagrangian = sympy.solve([k_max_density_enclosure_sphere, nucleus_separation_gradient[0] - lagrangian * constraint_gradient[0],  nucleus_separation_gradient[1] - lagrangian * constraint_gradient[1], nucleus_separation_gradient[2] - lagrangian * constraint_gradient[2]], x_pos, y_pos, z_pos, lagrangian, dict = True)
                    x_min = min_point_and_lagrangian[0][x_pos]
                    lagrangian = min_point_and_lagrangian[0][lagrangian]
                    x_min_chars = str(x_min)
                    if "x" in x_min_chars or "y" in x_min_chars or "z" in x_min_chars:
                        guess_vectors = 0
                        while guess_vectors < 1:
                            initial_x = np.random.uniform(pos_matrix[single_el_wavefunctions[k+1]-1][0],  pos_matrix[single_el_wavefunctions[k+1]-1][0] + maximum_el_density_radii[k])
                            initial_y = np.random.uniform(pos_matrix[single_el_wavefunctions[k+1]-1][1],  pos_matrix[single_el_wavefunctions[k+1]-1][1] + maximum_el_density_radii[k])
                            initial_density_enclosure_sphere = k_max_density_enclosure_sphere.replace(x_pos, initial_x).replace(y_pos, initial_y)
                            initial_z = sympy.solve(initial_density_enclosure_sphere, z_pos)
                            if len(initial_z)== 0:
                                continue
                            else:
                                guess_vector = np.array([initial_x, initial_y, initial_z[0]])
                                guess_vectors += 1
                        kth_axial_angle = sympy.symbols("theta", real = True)
                        k_theta = sympy.solve(sympy.tan(kth_axial_angle) - (initial_y / initial_x))[0]
                        k_azimuthl_angle = sympy.symbols("phi", real = True)
                        k_phi = sympy.solve(k_azimuthl_angle - sympy.acos(initial_z[0] / (sympy.sqrt(initial_x**2 + initial_y**2 + initial_z[0]**2))))[0]
                        k_theta_mirror = k_theta + np.pi
                        k_phi_mirror = np.pi - k_phi
                        x_mirror = maximum_el_density_radii[k] * sympy.sin(k_phi_mirror) * sympy.cos(k_theta_mirror)
                        y_mirror = maximum_el_density_radii[k] * sympy.sin(k_phi_mirror) * sympy.sin(k_theta_mirror)
                        z_mirror = maximum_el_density_radii[k] * sympy.cos(k_phi_mirror)
                        test_point_radius = (sympy.sqrt((x_mirror - initial_x)**2 + (y_mirror - initial_y)**2 + (z_mirror - initial_z[0])**2)) / 2
                        if abs(test_point_radius - maximum_el_density_radii[k]) < 1e-10:
                            kth_density_radius_scaling_factor = test_point_radius / nuclear_separation_matrix[single_el_wavefunctions[j+1]-1][single_el_wavefunctions[k+1]-1]
                            x_min = (pos_matrix[k][0] - pos_matrix[j][0]) * kth_density_radius_scaling_factor
                            y_min = (pos_matrix[k][1] - pos_matrix[j][1]) * kth_density_radius_scaling_factor
                            z_min = (pos_matrix[k][2] - pos_matrix[j][2]) * kth_density_radius_scaling_factor
                        if sympy.sqrt(x_min**2 + y_min**2 + z_min**2) < maximum_el_density_radii[j]:
                            overlap_state = 1
                            domain_overlap_matrix[int(j/2)][int(k/2)] = overlap_state
                            j_k_inner_overlap_cutoff = sympy.sqrt(x_min**2 + y_min**2 + z_min**2)
                            j_k_outer_overlap_cutoff = maximum_el_density_radii[j]
                            inner_overlap_cutoff_matrix[int(j/2)][int(2/k)] = j_k_inner_overlap_cutoff
                            outer_overlap_cutoff_matrix[int(j/2)][int(k/2)] = j_k_outer_overlap_cutoff
                            print(overlap_state, j_k_inner_overlap_cutoff, j_k_outer_overlap_cutoff)
                        else:
                            overlap_state = 0
                            domain_overlap_matrix[int(j/2)][int(k/2)] = overlap_state 
                            print(overlap_state)
                    else:
                        y_min = min_point_and_lagrangian[0][y_pos]
                        z_min = min_point_and_lagrangian[0][z_pos]
                        if jth_nucleus_separation_function.replace(x_pos, x_min).replace(y_pos, y_min).replace(z_pos, z_min) < maximum_el_density_radii[j]:
                            overlap_state = 1
                            domain_overlap_matrix[int(j/2)][int(k/2)] = overlap_state
                            j_k_inner_overlap_cutoff = jth_nucleus_separation_function.replace(x_pos, x_min).replace(y_pos, y_min).replace(z_pos, z_min)
                            j_k_outer_overlap_cutoff = maximum_el_density_radii[j]
                            inner_overlap_cutoff_matrix[int(j/2)][int(k/2)] = j_k_inner_overlap_cutoff
                            outer_overlap_cutoff_matrix[int(j/2)][int(k/2)] = j_k_outer_overlap_cutoff
                            print(overlap_state, j_k_inner_overlap_cutoff, j_k_outer_overlap_cutoff)
                        else:
                            overlap_state = 0
                            domain_overlap_matrix[int(j/2)][int(k/2)] = overlap_state
                            print(overlap_state)
                k += 2
            j += 2
        return maximum_el_density_radii, domain_overlap_matrix, inner_overlap_cutoff_matrix, outer_overlap_cutoff_matrix

    def slater_determinant(self, input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, n_bonding_electrons):
        single_el_wavefunctions, single_el_wavefunction_eqs, r = self.construct_single_electron_wavefunctions(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el)
        i = 0
        current_wavefunction_combinations = np.array([])
        wavefunction_combination_matrix = np.array([])
        electron_set = np.array([])
        wavefunc_set = np.array([])
        lower_diag = np.array([])
        while i < len(single_el_wavefunctions):
            wavefunc_set = np.append(wavefunc_set, single_el_wavefunctions[i])
            i += 2
        for j in np.arange(0, len(wavefunc_set)):
            for k in np.arange(0, n_bonding_electrons):
                current_electron = sympy.symbols("r" + str(k), real = True)
                current_wavefunction_combinations = np.append(current_wavefunction_combinations, wavefunc_set[j].replace(r, current_electron))
                k += 1
            j += 1
        wavefunction_combination_matrix = current_wavefunction_combinations.reshape(len(wavefunc_set), n_bonding_electrons)
        wavefunction_combination_matrix = sympy.Matrix(wavefunction_combination_matrix)
        perm, lower, upper = wavefunction_combination_matrix.LUdecomposition()
        perm = np.array(perm)
        lower = np.array(lower)
        upper = np.array(upper)
        print(lower)
        if len(perm) > 0:
            perm_diagonal_swaps = len(np.diag(perm)) - np.sum(np.diag(perm)) - 1
        else:
            perm_diagonal_swaps = 0
        #for i in np.arange(0, len(lower)):
            #print(lower[i][i])
            #lower_diag = np.append(lower_diag, lower[i][i])
            #print(i)
        #perm_det = (-1)**perm_diagonal_swaps
        #lower_det = np.prod(np.diag(lower))
        #upper_det = np.prod(np.diag(upper))
        #overall_system_wavefunc = upper_det * lower_det * perm_det
        return lower[0][0], lower[1][1], lower[2][2], lower[3][3], lower[4][4], lower[5][5]
        

    def MC_functional_integration_estimate(self, input_tensor, n_atoms, atomic_orbital_states, C_i_C_n, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, delta_o_cut, maximum_density_bound, nuclear_separation_matrix, bond_separation_matrix, atomic_positions, aromatic_rings, aromatic_cages, orbital_tracker, KS_functional_tracker, pos_matrix):  
        single_electron_wavefunctions, single_electron_eqns, r = self.construct_single_electron_wavefunctions(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el)
        r_eff_max, domain_overlap_matrix, inner_overlap_matrix, outer_overlap_matrix = self.compute_effective_density_cutoffs(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, maximum_density_bound, pos_matrix)
        BCP_bias_decay = -0.1
        # implement either straight-line distance of random var point on sphere from zero-gradient point of single-electron wavefunction, or staight-line distance from random R3 point from R3 line separating nuclei in bond
        MC_estimate = 0
        n_random_samples = 1000
        gradient_eval_samples = 500 #for evaluation of electron density gradient at every pm of the internuclear/ring/cage max effective separation
        sample_count = 0
        prev_gradient_sample = 0
        atomic_numbers = input_tensor.transpose()[2]
        a = 2 * orbital_tracker
        print(a)
        if KS_functional_tracker == "v_ext":
            while a <= (2 * orbital_tracker) + 1:
                max_density_separations = np.array([])
                random_pos_samples = np.array([])
                known_rings = np.array([])
                known_cages = np.array([])
                point_pdf_score = np.array([])
                nucleus_a_inner_cutoffs = np.array([])
                corrected_nuclear_separations = np.array([])
                for j in np.arange(0, len(aromatic_rings)):
                    if single_electron_wavefunctions[a+1] in aromatic_rings[j]:
                        nucleus_on_ring = True
                        electron_i_aromatic_ring = j
                        known_rings = np.append(known_rings, j)
                    else:
                        nucleus_on_ring = False
                for k in np.arange(0, len(aromatic_cages)):
                    if single_electron_wavefunctions[a+1] in aromatic_cages[j]:
                        nucleus_on_cage = True
                        electron_i_cage_surface = j
                        known_cages = np.append(known_cages, j)
                    else:
                        nucleus_on_cage = False
                if nucleus_on_ring == True:
                    RCPs = np.array([])
                    ring_gradient_changes = []
                    ring_atom_separations = np.array([])
                    ring_density_gradient_step = 0
                    nuclei_on_ring = aromicatic_rings[electron_i_aromatic_ring]
                    for k in np.arange(0, len(nuclei_on_ring)):
                        ring_atom_separations = np.append(ring_atom_separations, np.sqrt((pos_matrix[nuclei_on_ring[k]][0] - pos_matrix[single_electron_wavefunctions[a+1] - 1][0])**2 + (pos_matrix[nuclei_on_ring[k]][1] - pos_matrix[single_electron_wavefunctions[a+1] - 1][1])**2 + (pos_matrix[nuclei_on_ring[k]][2] - pos_matrix[single_electron_wavefunctions[a+1] - 1][2])**2))
                    adjusted_max_ring_pos = max(ring_atom_separations) - 2*R_inner
                    ring_density_gradient_step = ring_density_gradient_step + (adjusted_max_ring_pos / gradient_eval_samples)
                    j = 0
                    ring_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        ring_density_gradient_eval_set = np.append(ring_density_gradient_eval_set, R_inner + j * ring_density_gradient_step)
                        j += 1
                    j = 0
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, ring_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        ring_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(ring_gradient_changes)))
                    gradient_min_separation = ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(ring_atom_separations) - next_max_pos)]) < R_inner):
                        RCPs = np.append(RCPs, gradient_min_separation)
                    possible_density_maxima = np.array([R_inner])
                    possible_density_maxima = np.append(possible_density_maxima, RCPs)
                    for j in np.arange(0, len(ring_atom_separations)):
                        possible_density_maxima = np.append(possible_density_maxima, ring_atom_separations[j] - R_inner)
                    least_separation_from_density_max = min(possible_density_maxima)
                elif nucleus_on_cage == True:
                    CCPs = np.array([])
                    cage_gradient_changes = []
                    cage_atom_separations = np.array([])
                    cage_density_gradient_step = 0
                    nuclei_on_cage = aromatic_cages[electron_i_cage_surface]
                    for k in np.arange(0, len(nuclei_on_cage)):
                        cage_atom_separations = np.append(cage_atom_separations, np.sqrt((pos_matrix[nuclei_on_cage[j]][0] - pos_matrix[single_electron_wavefunctions[a+1] - 1][0])**2 + (pos_matrix[nuclei_on_cage[j]][1] - pos_matrix[single_electron_wavefunctions[a+1] - 1][1])**2 + (pos_matrix[nuclei_on_cage[j]][2] - pos_matrix[single_electron_wavefunctions[a+1] - 1][2])**2))
                    adjusted_max_cage_pos = max(cage_atom_separations) - 2*R_inner
                    cage_density_gradient_step = cage_densiy_gradient_step + (adjusted_max_cage_pos / gradient_eval_samples)
                    j = 0
                    cage_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        cage_density_gradient_eval_set = np.append(cage_density_gradient_eval_set, R_inner + j * cage_density_gradient_step)
                        j += 1
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    j = 0
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, cage_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        cage_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(cage_gradient_changes)))
                    gradient_min_separation = cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(cage_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(cage_atom_separations) - next_max_pos)]) < R_inner):
                        CCPs = np.append(CCPs, gradient_min_separation)
                    possible_density_maxima = np.array([R_inner])
                    possible_density_maxima = np.append(possible_density_maxima, CCPs)
                    for j in np.arange(0, len(cage_atom_separations)):
                        possible_density_maxima = np.append(possible_density_maxima, cage_atom_separations[j] - R_inner)
                    least_separation_from_density_max = min(possible_density_maxima)
                else:
                    BCPs = np.array([])
                    bond_gradient_changes = []
                    bond_density_gradient_step = 0
                    bond_maximum_separation = max(bond_separation_matrix[single_electron_wavefunctions[a+1] - 1]) 
                    adjusted_bond_max_separation = bond_maximum_separation - 2*R_inner
                    bond_density_gradient_step = bond_density_gradient_step + (adjusted_bond_max_separation / gradient_eval_samples)
                    j = 0
                    bond_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        bond_density_gradient_eval_set = np.append(bond_density_gradient_eval_set, R_inner + j * bond_density_gradient_step)
                        j += 1
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    j = 0
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, bond_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        bond_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(bond_gradient_changes)))
                    gradient_min_separation = bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(bond_maximum_separation - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(bond_maximum_separation - next_max_pos)]) < R_inner):
                        BCPs = np.append(BCPs, gradient_min_separation)
                    possible_density_maxima = np.array([R_inner])
                    possible_density_maxima = np.append(possible_density_maxima, BCPs)
                    for j in np.arange(0, len(bond_separation_matrix[single_electron_wavefunctions[a+1] - 1])):
                        if bond_separation_matrix[single_electron_wavefunctions[a+1] - 1][j] != 0:
                            possible_density_maxima = np.append(possible_density_maxima, bond_separation_matrix[single_electron_wavefunctions[a+1] - 1][j] - R_inner)
                        else:
                            continue
                ath_effective_density_limit = r_eff_max[a]
                directional_effective_density_limit = sympy.symbols("x_eff", real = True)
                directional_max_expression = sympy.sqrt(3 *  directional_effective_density_limit**2) - r_eff_max[a]
                directional_density_limit = np.abs(np.array(sympy.solvers.solve(directional_max_expression, directional_effective_density_limit)))
                for j in np.arange(0, len(nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1])):
                    if nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j] != 0:
                        nucleus_a_inner_cutoffs = np.append(nucleus_a_inner_cutoffs, nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j] - R_inner)
                        corrected_nuclear_separations = np.append(corrected_nuclear_separations, nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j])
                while sample_count < n_random_samples:
                    random_x = np.random.uniform(-directional_density_limit[0], directional_density_limit[0])
                    random_y = np.random.uniform(-directional_density_limit[0], directional_density_limit[0])
                    random_z = np.random.uniform(-directional_density_limit[0], directional_density_limit[0])
                    for j in np.arange(0, len(nucleus_a_inner_cutoffs)):
                        if nucleus_a_inner_cutoffs[j] < np.sqrt(random_x**2 + random_y**2 + random_z**2) < corrected_nuclear_separations[j]:
                            inner_cutoff_flag = True
                        else:
                            inner_cutoff_flag = False
                    if inner_cutoff_flag == True:
                        continue
                    elif np.sqrt(random_x**2 + random_y**2 + random_z**2) >= R_inner and np.sqrt(random_x**2 + random_y**2 + random_z**2) <= r_eff_max[a]:
                        random_pos_samples = np.append(random_pos_samples, (random_x**2 + random_y**2 + random_z**2)**(1/2))  
                        sample_count += 1
                    else:
                        continue
                max_density_separations = np.array([])
                for j in np.arange(0, len(random_pos_samples)):
                    for k in np.arange(0, len(possible_density_maxima)):
                        max_density_separations = np.append(max_density_separations, abs(random_pos_samples[j] - possible_density_maxima[k]))
                max_density_separations = max_density_separations.reshape(len(random_pos_samples), len(possible_density_maxima))
                for j in np.arange(0, len(max_density_separations)):
                    point_pdf_score = np.append(point_pdf_score, min(max_density_separations[j]))
                for j in np.arange(0, n_atoms):
                    v_ext = ath_electron_density_surface * (-atomic_numbers[j] / abs(atomic_positions[j] - r))
                    wavefunction_domain_volume= (4/3) * np.pi * ath_effective_density_limit**3
                for k in np.arange(0, n_random_samples):
                    BCP_bias = sympy.exp(BCP_bias_decay * point_pdf_score[k])
                    PDF = BCP_bias / wavefunction_domain_volume
                    KS_functional_sample = v_ext.replace(r, random_pos_samples[k])
                    MC_estimate = MC_estimate + KS_functional_sample
                MC_integral_approx = ((1 / PDF) / n_random_samples) + MC_estimate
                print(MC_integral_approx)
                a += 2
        elif KS_functional_tracker == "v_coul" or KS_functional_tracker == "v_xc":
            while a <= (2 * orbital_tracker) + 1:
                a_max_density_separations = np.array([])
                a_random_samples = np.array([])
                a_known_rings = np.array([])
                a_known_cages = np.array([])
                a_point_pdf_score = np.array([])
                nucleus_a_inner_cutoffs = np.array([])
                bond_a_corrected_nuclear_separations = np.array([])
                for j in np.arange(0, len(aromatic_rings)):
                    if single_electron_wavefunctions[a+1] in aromatic_rings[j]:
                        nucleus_on_ring = True
                        electron_i_aromatic_ring = j
                        a_known_rings = np.append(a_known_rings, j)
                    else:
                        nucleus_on_ring = False
                for k in np.arange(0, len(aromatic_cages)):
                    if single_electron_wavefunctions[a+1] in aromatic_cages[j]:
                        nucleus_on_cage = True
                        electron_i_cage_surface = j
                        a_known_cages = np.append(a_known_cages, j)
                    else:
                        nucleus_on_cage = False
                if nucleus_on_ring == True:
                    a_RCPs = np.array([])
                    a_ring_gradient_changes = []
                    a_ring_atom_separations = np.array([])
                    ring_density_gradient_step = 0
                    nuclei_on_a_ring = aromicatic_rings[electron_i_aromatic_ring]
                    for k in np.arange(0, len(nuclei_on_a_ring)):
                        a_ring_atom_separations = np.append(a_ring_atom_separations, np.sqrt((pos_matrix[nuclei_on_a_ring[k]][0] - pos_matrix[single_electron_wavefunctions[a+1] - 1][0])**2 + (pos_matrix[nuclei_on_a_ring[k]][1] - pos_matrix[single_electron_wavefunctions[a+1] - 1][1])**2 + (pos_matrix[nuclei_on_a_ring[k]][2] - pos_matrix[single_electron_wavefunctions[a+1] - 1][2])**2))
                    adjusted_max_a_ring_pos = max(a_ring_atom_separations) - 2*R_inner
                    ring_density_gradient_step = ring_density_gradient_step + (adjusted_max_a_ring_pos / gradient_eval_samples)
                    j = 0
                    a_ring_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        a_ring_density_gradient_eval_set = np.append(a_ring_density_gradient_eval_set, R_inner + j * ring_density_gradient_step)
                        j += 1
                    j = 0
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, a_ring_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        a_ring_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(a_ring_gradient_changes)))
                    gradient_min_separation = a_ring_density_gradient_eval_set[a_ring_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, a_ring_density_gradient_eval_set[a_ring_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, a_ring_density_gradient_eval_set[a_ring_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, a_ring_density_gradient_eval_set[a_ring_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(ring_atom_separations) - next_max_pos)]) < R_inner):
                        a_RCPs = np.append(a_RCPs, gradient_min_separation)
                elif nucleus_on_cage == True:
                     a_CCPs = np.array([])
                    a_cage_gradient_changes = []
                    a_cage_atom_separations = np.array([])
                    cage_density_gradient_step = 0
                    nuclei_on_a_cage = aromicatic_cages[electron_i_aromatic_cage]
                    for k in np.arange(0, len(nuclei_on_a_cage)):
                        a_cage_atom_separations = np.append(a_cage_atom_separations, np.sqrt((pos_matrix[nuclei_on_a_cage[k]][0] - pos_matrix[single_electron_wavefunctions[a+1] - 1][0])**2 + (pos_matrix[nuclei_on_a_cage[k]][1] - pos_matrix[single_electron_wavefunctions[a+1] - 1][1])**2 + (pos_matrix[nuclei_on_a_cage[k]][2] - pos_matrix[single_electron_wavefunctions[a+1] - 1][2])**2))
                    adjusted_max_a_cage_pos = max(a_cage_atom_separations) - 2*R_inner
                    cage_density_gradient_step = cage_density_gradient_step + (adjusted_max_a_cage_pos / gradient_eval_samples)
                    j = 0
                    a_cage_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        a_cage_density_gradient_eval_set = np.append(a_cage_density_gradient_eval_set, R_inner + j * cage_density_gradient_step)
                        j += 1
                    j = 0
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, a_cage_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        a_cage_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(a_cage_gradient_changes)))
                    gradient_min_separation = a_cage_density_gradient_eval_set[a_cage_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, a_cage_density_gradient_eval_set[a_cage_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, a_cage_density_gradient_eval_set[a_cage_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, a_cage_density_gradient_eval_set[a_cage_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(ring_atom_separations) - next_max_pos)]) < R_inner):
                        a_CCPs = np.append(a_CCPs, gradient_min_separation)
                else:
                    a_BCPs = np.array([])
                    a_bond_gradient_changes = []
                    a_bond_density_gradient_step = 0
                    a_bond_maximum_separation = max(bond_separation_matrix[single_electron_wavefunctions[a+1] - 1]) 
                    adjusted_a_bond_max_separation = a_bond_maximum_separation - 2*R_inner
                    bond_density_gradient_step = bond_density_gradient_step + (adjusted_a_bond_max_separation / gradient_eval_samples)
                    j = 0
                    a_bond_density_gradient_eval_set = np.array([R_inner])
                    while j <= gradient_eval_samples:
                        a_bond_density_gradient_eval_set = np.append(a_bond_density_gradient_eval_set, R_inner + j * bond_density_gradient_step)
                        j += 1
                    ath_electron_density_surface = single_electron_wavefunctions[a]**2
                    ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                    j = 0
                    while j <= gradient_eval_samples:
                        current_gradient_sample = ath_electron_density_gradient.replace(r, a_bond_density_gradient_eval_set[j])
                        current_gradient_change = current_gradient_sample - prev_gradient_sample
                        a_bond_gradient_changes.append(current_gradient_change)
                        prev_gradient_sample = current_gradient_sample
                        j += 1
                    gradient_min = min(np.abs(np.array(a_bond_gradient_changes)))
                    gradient_min_separation = a_bond_density_gradient_eval_set[a_bond_gradient_changes.index(gradient_min)]
                    preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                    next_density_vals = [ath_electron_density_surface.replace(r, a_bond_density_gradient_eval_set[a_bond_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, a_bond_density_gradient_eval_set[a_bond_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, a_bond_density_gradient_eval_set[a_bond_gradient_changes.index(gradient_min)] + 0.03)]
                    preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                    next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                    preceeding_max = max(preceeding_density_vals)
                    next_max = max(next_density_vals)
                    preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                    next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                    if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(bond_maximum_separation - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(bond_maximum_separation - next_max_pos)]) < R_inner):
                        a_BCPs = np.append(a_BCPs, gradient_min_separation)
                if len(a_RCPs) > 0:
                    a_possible_density_maxima = np.array([R_inner])
                    a_possible_density_maxima = np.append(possible_density_maxima, a_RCPs)
                    for j in np.arange(0, len(a_ring_atom_separations)):
                        a_possible_density_maxima = np.append(a_possible_density_maxima, a_ring_atom_separations[j] - R_inner)
                    least_separation_from_a_density_max = min(a_possible_density_maxima)
                elif len(a_CCPs) > 0:
                    a_possible_density_maxima = np.array([R_inner])
                    a_possible_density_maxima = np.append(a_possible_density_maxima, a_CCPs)
                    for j in np.arange(0, len(a_cage_atom_separations)):
                        a_possible_density_maxima = np.append(a_possible_density_maxima, a_cage_atom_separations[j] - R_inner)
                    least_separation_from_a_density_max = min(a_possible_density_maxima)
                else:
                    a_possible_density_maxima = np.array([R_inner])
                    a_possible_density_maxima = np.append(a_possible_density_maxima, a_BCPs)
                    for j in np.arange(0, len(bond_separation_matrix[single_electron_wavefunctions[a+1] - 1])):
                        if bond_separation_matrix[single_electron_wavefunctions[a+1] - 1][j] != 0:
                            a_possible_density_maxima = np.append(a_possible_density_maxima, bond_separation_matrix[single_electron_wavefunctions[a+1] - 1][j] - R_inner)
                        else:
                            continue  
                ath_effective_density_limit = r_eff_max[a]
                ath_directional_effective_density_limit = sympy.symbols("x_eff_a", real = True)
                ath_directional_max_expression = sympy.sqrt(3 *  ath_directional_effective_density_limit**2) - r_eff_max[a]
                ath_directional_density_limit = np.abs(np.array(sympy.solvers.solve(ath_directional_max_expression, ath_directional_effective_density_limit)))
                for j in np.arange(0, len(nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1])):
                    if nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j] != 0:
                        nucleus_a_inner_cutoffs = np.append(nucleus_a_inner_cutoffs, nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j] - R_inner)
                        bond_a_corrected_nuclear_separations = np.append(bond_a_corrected_nuclear_separations, nuclear_separation_matrix[single_electron_wavefunctions[a+1]-1][j])
                b = 0
                while b < len(n_bonding_electrons):
                    if b != a:
                        b_max_density_separations = np.array([])
                        b_random_samples = np.array([])
                        b_point_pdf_score = np.array([])
                        nucleus_b_inner_cutoffs = np.array([])
                        bond_b_corrected_nuclear_separations = np.array([])
                        for j in np.arange(0, len(aromatic_rings)):
                            if single_electron_wavefunctions[2*b+1] in aromatic_rings[j]:
                                nucleus_on_ring = True
                                electron_i_aromatic_ring = j
                                known_rings = np.append(known_rings, j)
                            else:
                                nucleus_on_ring = False
                        for k in np.arange(0, len(aromatic_cages)):
                            if single_electron_wavefunctions[a+1] in aromatic_cages[j]:
                                nucleus_on_cage = True
                                electron_i_cage_surface = j
                                known_cages = np.append(known_cages, j)
                            else:
                                nucleus_on_cage = False
                        if nucleus_on_ring == True:
                            if known_rings[0] == a_known_rings[0]:
                                b_RCPs = a_RCPs
                            else:
                                b_RCPs = np.array([])
                                b_ring_gradient_changes = []
                                b_ring_atom_separations = np.array([])
                                ring_density_gradient_step = 0
                                nuclei_on_b_ring = aromicatic_rings[electron_i_aromatic_ring]
                                for k in np.arange(0, len(nuclei_on_b_ring)):
                                    b_ring_atom_separations = np.append(b_ring_atom_separations, np.sqrt((pos_matrix[nuclei_on_b_ing[k]][0] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][0])**2 + (pos_matrix[nuclei_on_b_ring[k]][1] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][1])**2 + (pos_matrix[nuclei_on_b_ring[k]][2] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][2])**2))
                                adjusted_max_b_ring_pos = max(b_ring_atom_separations) - 2*R_inner
                                ring_density_gradient_step = ring_density_gradient_step + (adjusted_max_b_ring_pos / gradient_eval_samples)
                                j = 0
                                b_ring_density_gradient_eval_set = np.array([R_inner])
                                while j <= gradient_eval_samples:
                                    ring_density_gradient_eval_set = np.append(ring_density_gradient_eval_set, R_inner + j * ring_density_gradient_step)
                                    j += 1
                                j = 0
                                bth_electron_density_surface = single_electron_wavefunctions[2*b]**2
                                bth_electron_density_gradient = sympy.diff(bth_electron_density_surface, r)
                                while j <= gradient_eval_samples:
                                    current_gradient_sample = bth_electron_density_gradient.replace(r, ring_density_gradient_eval_set[j])
                                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                                    b_ring_gradient_changes.append(current_gradient_change)
                                    prev_gradient_sample = current_gradient_sample
                                    j += 1
                                gradient_min = min(np.abs(np.array(b_ringring_gradient_changes)))
                                gradient_min_separation = b_ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)]
                                preceeding_density_vals = [bth_electron_density_surface.replace(r, gradient_min_separation - 0.01), bth_electron_density_surface.replace(r, gradient_min_separation - 0.02), bth_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                                next_density_vals = [bth_electron_density_surface.replace(r, b_ring_density_gradient_eval_set[b_ring_gradient_changes.index(gradient_min)] + 0.01), bth_electron_density_surface.replace(r, b_ring_density_gradient_eval_set[b_ring_gradient_changes.index(gradient_min)] + 0.02), bth_electron_density_surface.replace(r, b_ring_density_gradient_eval_set[b_ring_gradient_changes.index(gradient_min)] + 0.03)]
                                preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                                preceeding_max = max(preceeding_density_vals)
                                next_max = max(next_density_vals)
                                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                                if (bth_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(b_ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (bth_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(b_ring_atom_separations) - next_max_pos)]) < R_inner):
                                    b_RCPs = np.append(b_RCPs, gradient_min_separation)
                        elif nucleus_on_cage == True:
                            if known_cages[0] == a_known_cages[0]:
                                b_CCPs = a_CCPs
                            else:
                                b_CCPs = np.array([])
                                b_cage_gradient_changes = []
                                b_cage_atom_separations = np.array([])
                                cage_density_gradient_step = 0
                                nuclei_on_b_cage = aromicatic_rings[electron_i_aromatic_cage]
                                for k in np.arange(0, len(nuclei_on_b_cage)):
                                    b_cage_atom_separations = np.append(b_cage_atom_separations, np.sqrt((pos_matrix[nuclei_on_b_cage[k]][0] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][0])**2 + (pos_matrix[nuclei_on_b_cage[k]][1] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][1])**2 + (pos_matrix[nuclei_on_b_cage[k]][2] - pos_matrix[single_electron_wavefunctions[2*b+1] - 1][2])**2))
                                adjusted_max_b_cage_pos = max(b_cage_atom_separations) - 2*R_inner
                                cage_density_gradient_step = cage_density_gradient_step + (adjusted_max_b_cage_pos / gradient_eval_samples)
                                j = 0
                                b_cage_density_gradient_eval_set = np.array([R_inner])
                                while j <= gradient_eval_samples:
                                    cage_density_gradient_eval_set = np.append(cage_density_gradient_eval_set, R_inner + j * cage_density_gradient_step)
                                    j += 1
                                j = 0
                                bth_electron_density_surface = single_electron_wavefunctions[2*b]**2
                                bth_electron_density_gradient = sympy.diff(bth_electron_density_surface, r)
                                while j <= gradient_eval_samples:
                                    current_gradient_sample = bth_electron_density_gradient.replace(r, cage_density_gradient_eval_set[j])
                                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                                    b_cage_gradient_changes.append(current_gradient_change)
                                    prev_gradient_sample = current_gradient_sample
                                    j += 1
                                gradient_min = min(np.abs(np.array(b_cage_gradient_changes)))
                                gradient_min_separation = b_cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)]
                                preceeding_density_vals = [bth_electron_density_surface.replace(r, gradient_min_separation - 0.01), bth_electron_density_surface.replace(r, gradient_min_separation - 0.02), bth_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                                next_density_vals = [bth_electron_density_surface.replace(r, b_cage_density_gradient_eval_set[b_cage_gradient_changes.index(gradient_min)] + 0.01), bth_electron_density_surface.replace(r, b_cage_density_gradient_eval_set[b_cage_gradient_changes.index(gradient_min)] + 0.02), bth_electron_density_surface.replace(r, b_cage_density_gradient_eval_set[b_cage_gradient_changes.index(gradient_min)] + 0.03)]
                                preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                                preceeding_max = max(preceeding_density_vals)
                                next_max = max(next_density_vals)
                                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                                if (bth_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(b_cage_atom_separations) - preceeding_max_pos)]) < R_inner) and (bth_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(b_cage_atom_separations) - next_max_pos)]) < R_inner):
                                    b_CCPs = np.append(b_CCPs, gradient_min_separation)
                        else:
                            b_BCPs = np.array([])
                            b_bond_gradient_changes = []
                            b_bond_density_gradient_step = 0
                            b_bond_maximum_separation = max(bond_separation_matrix[single_electron_wavefunctions[2*b+1] - 1]) 
                            adjusted_b_bond_max_separation = b_bond_maximum_separation - 2*R_inner
                            bond_density_gradient_step = bond_density_gradient_step + (adjusted_b_bond_max_separation / gradient_eval_samples)
                            j = 0
                            b_bond_density_gradient_eval_set = np.array([R_inner])
                            while j <= gradient_eval_samples:
                                b_bond_density_gradient_eval_set = np.append(b_bond_density_gradient_eval_set, R_inner + j * bond_density_gradient_step)
                                j += 1
                            bth_electron_density_surface = single_electron_wavefunctions[a]**2
                            bth_electron_density_gradient = sympy.diff(bth_electron_density_surface, r)
                            j = 0
                            while j <= gradient_eval_samples:
                                current_gradient_sample = bth_electron_density_gradient.replace(r, abbond_density_gradient_eval_set[j])
                                current_gradient_change = current_gradient_sample - prev_gradient_sample
                                b_bond_gradient_changes.append(current_gradient_change)
                                prev_gradient_sample = current_gradient_sample
                                j += 1
                            gradient_min = min(np.abs(np.array(b_bond_gradient_changes)))
                            gradient_min_separation = b_bond_density_gradient_eval_set[b_bond_gradient_changes.index(gradient_min)]
                            preceeding_density_vals = [bth_electron_density_surface.replace(r, gradient_min_separation - 0.01), bth_electron_density_surface.replace(r, gradient_min_separation - 0.02), bth_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                            next_density_vals = [bth_electron_density_surface.replace(r, b_bond_density_gradient_eval_set[b_bond_gradient_changes.index(gradient_min)] + 0.01), bth_electron_density_surface.replace(r, b_bond_density_gradient_eval_set[b_bond_gradient_changes.index(gradient_min)] + 0.02), bth_electron_density_surface.replace(r, b_bond_density_gradient_eval_set[b_bond_gradient_changes.index(gradient_min)] + 0.03)]
                            preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                            next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                            preceeding_max = max(preceeding_density_vals)
                            next_max = max(next_density_vals)
                            preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                            next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                            if (bth_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(b_bond_maximum_separation - preceeding_max_pos)]) < R_inner) and (bth_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(b_bond_maximum_separation - next_max_pos)]) < R_inner):
                                b_BCPs = np.append(a_BCPs, gradient_min_separation)
                        if len(b_RCPs) > 0:
                            b_possible_density_maxima = np.array([R_inner])
                            b_possible_density_maxima = np.append(b_possible_density_maxima, b_RCPs)
                            for j in np.arange(0, len(b_ring_atom_separations)):
                                b_possible_density_maxima = np.append(b_possible_density_maxima, b_ring_atom_separations[j] - R_inner)
                            least_separation_from_b_density_max = min(b_possible_density_maxima)
                        elif len(b_CCPs) > 0:
                            b_possible_density_maxima = np.array([R_inner])
                            b_possible_density_maxima = np.append(b_possible_density_maxima, b_CCPs)
                            for j in np.arange(0, len(b_cage_atom_separations)):
                                b_possible_density_maxima = np.append(b_possible_density_maxima, b_cage_atom_separations[j] - R_inner)
                            least_separation_from_b_density_max = min(b_possible_density_maxima)
                        else:
                            b_possible_density_maxima = np.array([R_inner])
                            b_possible_density_maxima = np.append(b_possible_density_maxima, b_BCPs)
                            for j in np.arange(0, len(bond_separation_matrix[single_electron_wavefunctions[2*b+1] - 1])):
                                if bond_separation_matrix[single_electron_wavefunctions[2*b+1] - 1][j] != 0:
                                    b_possible_density_maxima = np.append(b_possible_density_maxima, bond_separation_matrix[single_electron_wavefunctions[2*b+1] - 1][j] - R_inner)
                                else:
                                    continue 
                    bth_effective_density_limit = r_eff_max[b]
                    bth_directional_effective_density_limit = sympy.symbols("x_eff_b", real = True)
                    bth_directional_max_expression = sympy.sqrt(3 *  bth_directional_effective_density_limit**2) - r_eff_max[b]
                    bth_directional_density_limit = np.abs(np.array(sympy.solvers.solve(bth_directional_max_expression, bth_directional_effective_density_limit)))
                    for j in np.arange(0, len(nuclear_separation_matrix[single_electron_wavefunctions[2*b+1]-1])):
                        if nuclear_separation_matrix[single_electron_wavefunctions[2*b+1]-1][j] != 0:
                            nucleus_b_inner_cutoffs = np.append(nucleus_b_inner_cutoffs, nuclear_separation_matrix[single_electron_wavefunctions[2*b+1]-1][j] - R_inner)
                            bond_b_corrected_nuclear_separations = np.append(bond_b_corrected_nuclear_separations, nuclear_separation_matrix[single_electron_wavefunctions[2*b+1]-1][j])
                    if domain_overlap_matrix[a][b] == 0:
                        while sample_count < n_random_samples:
                            a_random_x = np.random.uniform(-ath_directional_density_limit[0], ath_directional_density_limit[0])
                            a_random_y = np.random.uniform(-ath_directional_density_limit[0], ath_directional_density_limit[0])
                            a_random_z = np.random.uniform(-ath_directional_density_limit[0], ath_directional_density_limit[0])
                        for j in np.arange(0, len(nucleus_a_inner_cutoffs)):
                            if nucleus_a_inner_cutoffs[j] < np.sqrt(a_random_x**2 + a_random_y**2 + a_random_z**2) < bond_a_corrected_nuclear_separations[j]:
                                inner_cutoff_flag = True
                        else:
                                inner_cutoff_flag = False
                        if inner_cutoff_flag == True:
                            continue
                        elif np.sqrt(a_random_x**2 + a_random_y**2 + a_random_z**2) >= R_inner and np.sqrt(a_random_x**2 + a_random_y**2 + a_random_z**2) <= r_eff_max[a]:
                            a_random_samples = np.append(a_random_samples, (a_random_x**2 + a_random_y**2 + a_random_z**2)**(1/2))  
                            sample_count += 1
                        else:
                            continue
                        sample_count = 0
                        while sample_count < n_random_samples:
                            b_random_x = np.random.uniform(-bth_directional_density_limit[0], bth_directional_density_limit[0])
                            b_random_y = np.random.uniform(-bth_directional_density_limit[0], bth_directional_density_limit[0])
                            b_random_z = np.random.uniform(-bth_directional_density_limit[0], bth_directional_density_limit[0])
                        for j in np.arange(0, len(nucleus_b_inner_cutoffs)):
                            if nucleus_b_inner_cutoffs[j] < np.sqrt(b_random_x**2 + b_random_y**2 + b_random_z**2) < bond_b_corrected_nuclear_separations[j]:
                            inner_cutoff_flag = True
                        else:
                            inner_cutoff_flag = False
                        if inner_cutoff_flag == True:
                            continue
                        elif np.sqrt(b_random_x**2 + b_random_y**2 + b_random_z**2) >= R_inner and np.sqrt(b_random_x**2 + b_random_y**2 + b_random_z**2) <= r_eff_max[b]:
                            b_random_samples = np.append(b_random_samples, (b_random_x**2 + b_random_y**2 + b_random_z**2)**(1/2))  
                            sample_count += 1
                        else:
                            continue
                    else:
                        if inner_overlap_matrix[a][b] != 0:
                            ath_effective_density_limit = inner_overlap_matrix[a][b]
                            bth_effective_density_limit = bond_separation_matrix[a][b] - inner_overlap_matrix.transpose()[a][b]
                            ath_spherical_max_expression = sympy.sqrt(3 *  ath_directional_effective_density_limit**2) - r_eff_max[a]
                            ath_spherical_density_limit = np.abs(np.array(sympy.solvers.solve(bth_directional_max_expression, bth_directional_effective_density_limit)))
                            bth_spherical_max_expression = sympy.sqrt(3 *  bth_directional_effective_density_limit**2) - r_eff_max[b]
                            ath_unoverlapped_effective_limit = ath_spherical_density_limit * (ath_effective_density_limit / ath_spherical_density_limit)
                            bth_unoverlapped_effective_limit = bth_spherical_density_limit * (bth_effective_density_limit / bth_spherical_density_limit)
                            overlap_dir = np.random.uniform(1, 3)
                            sample_count = 0
                            while sample_count < n_random_samples:
                                if overlap_dir == 1:
                                    a_random_x = np.random.uniform(-ath_spherical_density_limit[0], ath_unoverlapped_effective_limit)
                                    a_random_y = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                    a_random_z = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                elif overlap_dir == 2:
                                    a_random_x = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                    a_random_y = np.random.uniform(-ath_spherical_density_limit[0], ath_unoverlapped_effective_limit)
                                    a_random_z = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                elif overlap_dir == 2:
                                    a_random_x = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                    a_random_y = np.random.uniform(-ath_spherical_density_limit[0], ath_spherical_density_limit[0])
                                    a_random_z = np.random.uniform(-ath_spherical_density_limit[0], ath_unoverlapped_effective_limit)
                max_density_separations = np.array([])
                for j in np.arange(0, len(random_pos_samples)):
                    for k in np.arange(0, len(possible_density_maxima)):
                        max_density_separations = np.append(max_density_separations, abs(random_pos_samples[j] - possible_density_maxima[k]))
                max_density_separations = max_density_separations.reshape(len(random_pos_samples), len(possible_density_maxima))
                for j in np.arange(0, len(max_density_separations)):
                    point_pdf_score = np.append(point_pdf_score, min(max_density_separations[j]))
                for j in np.arange(0, n_atoms):
                    v_ext = ath_electron_density_surface * (-atomic_numbers[j] / abs(atomic_positions[j] - r))
                    wavefunction_domain_volume= (4/3) * np.pi * ath_effective_density_limit**3
                for k in np.arange(0, n_random_samples):
                    BCP_bias = sympy.exp(BCP_bias_decay * point_pdf_score[k])
                    PDF = BCP_bias / wavefunction_domain_volume
                    KS_functional_sample = v_ext.replace(r, random_pos_samples[k])
                    MC_estimate = MC_estimate + KS_functional_sample
                MC_integral_approx = ((1 / PDF) / n_random_samples) + MC_estimate
                print(MC_integral_approx)
        return MC_integral_approx


    def compute_system_energy_eigenval(self, C_i_C_n, E_xc, n_bonding_electrons, n_atoms, unit_charge, R_inner, delta_o_cut, maximum_density_bound, input_tensor, atomic_orbital_states, exponential_range, nuclear_separation_matrix, bond_separation_matrix, degree_vector, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, orbital_tracker, KS_functional_tracker, pos_matrix):
        degree_vector = degree_vector.astype(int)
        degree_vector = np.insert(degree_vector, 0, 0)
        single_electron_wavefunctions, single_electron_wavefunc_eqns, r = self.construct_single_electron_wavefunctions(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el)
        r_prime = sympy.symbols("r_prime", real = True)
        kinetic_energy_operator = 0
        external_potential = 0
        exchange_correlation_potential = sympy.symbols("v_e_xc", real = True)
        electronic_repulsion_potential = sympy.symbols("v_el", real = True)
        contraction_coeff = sympy.symbols("Ci", real = True)
        k_symb_expressions = np.array([])
        atomic_numbers = input_tensor.transpose()[2]
        atomic_positions = input_tensor.transpose()[0]
        if n_bonding_electrons % 2 == 0:
            i = 0
            while i < n_bonding_electrons:
                laplacian = (1/r**2) * (sympy.diff(single_electron_wavefunctions[2*i], r) * (r**2 * sympy.diff(single_electron_wavefunctions[2*i], r)))
                ith_electron_density_surface = single_electron_wavefunctions[2*i]**2
                K = sympy.integrals.integrate(ith_electron_density_surface * laplacian, (r, R_inner, max(bond_separation_matrix[single_electron_wavefunctions[2*i+1] - 1]) + delta_o_cut))
                kinetic_energy_operator = kinetic_energy_operator + K
                print(kinetic_energy_operator)
                i += 1
            KS_functional_tracker = "v_ext"
            i = 0
            while i < n_bonding_electrons:
                orbital_tracker = i
                ith_el_external_potential = self.MC_functional_integration_estimate(input_tensor, n_atoms, atomic_orbital_states, C_i_C_n, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, delta_o_cut, maximum_density_bound, nuclear_separation_matrix, bond_separation_matrix, atomic_positions, aromatic_rings, aromatic_cages, orbital_tracker, KS_functional_tracker)
                external_potential = external_potential + ith_el_external_potential
                print(external_potential)
                i += 1
            KS_functional_tracker = "v_coul"
            i = 0
            while i < n_bonding_electrons:
                orbital_tracker = i
                ith_el_external_potential = self.MC_functional_integration_estimate(input_tensor, n_atoms, atomic_orbital_states, C_i_C_n, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, delta_o_cut, maximum_density_bound, nuclear_separation_matrix, bond_separation_matrix, atomic_positions, aromatic_rings, aromatic_cages, orbital_tracker, KS_functional_tracker)
                external_potential = external_potential + ith_el_external_potential
                print(external_potential)
                i += 1
            
        return kinetic_energy_operator, external_potential
        
                
        i = 0
        while i <= n_bonding_electrons:
            j = i + 2
            #while j <= n_bonding_electrons:
                #v_coul_min_char_list = []
                #v_coul_max_char_list = []
                #v_coul_min_replacement_str = ""
                #v_coul_max_replacement_str = ""
                #jth_electron_density_surface = single_electron_wavefunctions[j].replace(r, r_prime)**2
                #v_coul_electron_j = jth_electron_density_surface / abs(r - r_prime)
                #electron_j_series_approx = sympy.series(v_coul_electron_j)
                #v_coul_electron_j = sympy.integrals.integrate(electron_j_series_approx, r_prime)
                #v_coul_j_str = str(v_coul_electron_j)
                #k = 0
                #for k in range(0, len(v_coul_j_str)):
                    #v_coul_min_char_list.append(v_coul_j_str[k])
                    #v_coul_max_char_list.append(v_coul_j_str[k])
                    #k += 1
               # k = 0
                #for k in range(0, len(v_coul_min_char_list)):
                    #if v_coul_min_char_list[k] == "r_prime":
                        #v_coul_min_char_list[k] = str(R_inner)
                    #elif v_coul_min_char_list[k] == "O":
                        #v_coul_min_drop_index = k
                    #else:
                        #k += 1
                #del v_coul_min_char_list[v_coul_min_drop_index - 2:]
                #k = 0
                #for k in range(0, len(v_coul_max_char_list)):
                    #if v_coul_max_char_list[k] == "r":
                        #v_coul_max_char_list[k] = str(maximum_density_bound)
                    #elif v_coul_max_char_list[k] == "O":
                        #v_coul_max_drop_index = k
                    #else:
                        #k += 1
                #del v_coul_max_char_list[v_coul_max_drop_index - 2:]
                #k = 0
                #for k in range(0, len(v_coul_min_char_list)):
                    #v_coul_min_replacement_str =  v_coul_min_replacement_str + v_coul_min_char_list[k]
               # k = 0
                #for k in range(0, len(v_coul_max_char_list)):
                    #v_coul_max_replacement_str =  v_coul_max_replacement_str + v_coul_max_char_list[k]
                    #v_coul_electron_j_max = sympy.parsing.sympy_parser.parse_expr(v_coul_max_replacement_str)
                    #v_coul_electron_j_min = sympy.parsing.sympy_parser.parse_expr(v_coul_min_replacement_str)
                    #v_coul_electron_j = v_coul_electron_j_max - v_coul_electron_j_min
                    #j += 2
            #i += 2


            
                    #v_ext_i_placeholder = sympy.symbols("v_ext_i", real = True) 
                    #ith_el_external_potential = ith_el_external_potential + v_ext_i
                    #ith_el_external_potential = ith_el_external_potential.replace(v_ext_i_placeholder, 0)
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 663)

In [17]:
def compute_interatomic_separations(nuclear_coordinates, adj_matrix):
    nuclear_positions = pd.read_csv(nuclear_coordinates)
    pos_matrix = nuclear_positions.to_numpy()
    interatomic_distances = np.zeros_like(adj_matrix)
    i = 0
    j = 0
    for i in np.arange(0, len(pos_matrix)):
        for j in np.arange(0, len(pos_matrix)):
            if i == j:
                i_j_internuclear_separation = 0
                interatomic_distances[i][j] = i_j_internuclear_separation
            else:
                i_j_internuclear_separation = np.sqrt((pos_matrix[i][0] - pos_matrix[j][0])**2 + (pos_matrix[i][1] - pos_matrix[j][1])**2 + (pos_matrix[i][2] - pos_matrix[j][2])**2)
                interatomic_distances[i][j] = i_j_internuclear_separation
    return interatomic_distances

def compute_bond_separations(nuclear_coordinates, degree_vector, adj_matrix):
    nuclear_positions = pd.read_csv(nuclear_coordinates)
    pos_matrix = nuclear_positions.to_numpy()
    bond_sep_matrix = np.zeros_like(adj_matrix)
    i = 0
    j = 0
    for i in np.arange(0, len(pos_matrix)):
        if degree_vector[i] > 0:
            for j in np.arange(0, len(adj_matrix[i])):
                if adj_matrix[i][j] == 1:
                    i_j_internuclear_separation = np.sqrt((pos_matrix[j][0] - pos_matrix[i][0])**2 + (pos_matrix[j][1] - pos_matrix[i][1])**2 + (pos_matrix[j][2] - pos_matrix[i][2])**2)
                    bond_sep_matrix[i][j] = i_j_internuclear_separation
                j += 1
        i += 1
    bond_sep_list = list(bond_sep_matrix.flatten())
    for k in bond_sep_list:
        if abs(k) == 0:
            bond_sep_list.pop(bond_sep_list.index(k))
    bond_sep_set = np.array(bond_sep_list)
    return bond_sep_matrix, pos_matrix, bond_sep_set
                    
degree_vector = compute_node_connection_degrees(adj_matrix, n_atoms)
nuclear_coordinates = "/users/haydenprescott/documents/nuclear_pos_test.csv"
bond_separation_matrix, pos_matrix, bond_separation_set = compute_bond_separations(nuclear_coordinates, degree_vector, adj_matrix)
nuclear_separation_matrix = compute_interatomic_separations(nuclear_coordinates, adj_matrix)
print(nuclear_separation_matrix, bond_separation_matrix)

[[ 0.          2.49721845  3.10412629  5.4         1.9037857   2.70118493
   4.7       ]
 [ 2.49721845  0.          4.06148987  7.68427615  4.25775763  3.11173585
   2.78210352]
 [ 3.10412629  4.06148987  0.          4.63849113  4.32185146  5.76055553
   6.83853786]
 [ 5.4         7.68427615  4.63849113  0.          4.6887525   7.5625657
  10.1       ]
 [ 1.9037857   4.25775763  4.32185146  4.6887525   0.          2.93339394
   5.92574046]
 [ 2.70118493  3.11173585  5.76055553  7.5625657   2.93339394  0.
   3.36725407]
 [ 4.7         2.78210352  6.83853786 10.1         5.92574046  3.36725407
   0.        ]] [[0.         2.49721845 0.         0.         0.         2.70118493
  4.7       ]
 [2.49721845 0.         4.06148987 0.         0.         0.
  0.        ]
 [0.         4.06148987 0.         4.63849113 0.         0.
  6.83853786]
 [0.         0.         4.63849113 0.         4.6887525  0.
  0.        ]
 [0.         0.         0.         4.6887525  0.         2.93339394
  5.92574046]

In [18]:
print(nuclear_separation_matrix)

[[ 0.          2.49721845  3.10412629  5.4         1.9037857   2.70118493
   4.7       ]
 [ 2.49721845  0.          4.06148987  7.68427615  4.25775763  3.11173585
   2.78210352]
 [ 3.10412629  4.06148987  0.          4.63849113  4.32185146  5.76055553
   6.83853786]
 [ 5.4         7.68427615  4.63849113  0.          4.6887525   7.5625657
  10.1       ]
 [ 1.9037857   4.25775763  4.32185146  4.6887525   0.          2.93339394
   5.92574046]
 [ 2.70118493  3.11173585  5.76055553  7.5625657   2.93339394  0.
   3.36725407]
 [ 4.7         2.78210352  6.83853786 10.1         5.92574046  3.36725407
   0.        ]]


In [35]:



    def construct_grad_descent_schedule(self, LR_o, t, K, gradient_descent_schedule):
        # implement RMS normalization for stochastic gradient descent - calculate gradients as cumulative moving average of sqaures, normalize by square root of mean-square gradient, and update weights in direction of gloabl min in accordance with sign of previous gradient with respect to the previous weight 
        # paper link: https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a
        if gradient_descent_schedule == "MBGD":
            LR = LR_o * np.exp(-K * t)
        return LR
   
    def construct_single_electron_wavefunctions(self, input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el):
        s_MO_set = sympy.Array([])
        p_MO_set = sympy.Array([])
        d_MO_set = sympy.Array([])
        f_MO_set = sympy.Array([])
        s_ref_set = sympy.Array([])
        p_ref_set = sympy.Array([])
        d_ref_set = sympy.Array([])
        f_ref_set = sympy.Array([])
        s_exp_set = sympy.Array([])
        p_exp_set = sympy.Array([])
        d_exp_set = sympy.Array([])
        f_exp_set = sympy.Array([])
        exp_set = sympy.Array([])
        one_electron_MOs = np.array([])
        gaussian_terms = []
        coefficient_sequence = []
        exp_set = []
        r = sympy.symbols("r", real = True)
        j = sympy.symbols("j", real = True)
        s_exponentials = 1
        p_exponentials = atomic_orbital_states["p"][1] - atomic_orbital_states["s"]
        d_exponentials = atomic_orbital_states["d"][1] - atomic_orbital_states["p"][1]
        f_exponentials = atomic_orbital_states["f"][0]
        i = 0
        for i in range(0, n_atoms):
            if len(C_i_C_n.transpose()) > s_exponentials - 1:
                exp_start = exponential_range[s_exponentials - 1]
                s_coefficient_range = C_i_C_n[i][0:maximum_s_el * s_exponentials]
                j = 0
                for j in range(0, len(s_coefficient_range)):
                    current_contraction_coeff = s_coefficient_range[j]
                    if current_contraction_coeff != 0:
                        current_s_MO = current_contraction_coeff * sympy.exp(-input_tensor[i][exp_start] * r)
                        s_MO_set = np.append(s_MO_set, np.array([current_s_MO, i + 1]))
                        s_ref_set = np.append(s_ref_set, current_s_MO)
                        s_exp_set = np.append(s_exp_set, sympy.exp(-input_tensor[i][exp_start] * r))
                        gaussian_terms.append(current_s_MO)
                        coefficient_sequence.append(current_contraction_coeff)
                        j += s_exponentials
            else:
                pass
            exp_range_vals = np.arange(exponential_range[0], exponential_range[1] + 1)
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el:
                p_coefficient_range = C_i_C_n[i][maximum_s_el * s_exponentials : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials]
                j = 0
                p_MO_contributions = 0
                while j <= len(p_coefficient_range):
                    current_contraction_coeffs = p_coefficient_range[j: j + p_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["p"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            p_exp_set = np.append(p_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["p"][0]] * r))
                            p_MO_contributions = p_MO_contributions + current_gaussian_term
                            k += 1
                    j += p_exponentials
                    if j <= len(p_coefficient_range):
                        p_MO_set = np.append(p_MO_set, np.array([p_MO_contributions, i + 1]))  
                        p_ref_set = np.append(p_ref_set, p_MO_contributions)
            else:
                pass
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el + p_exponentials * maximum_p_el:
                d_coefficient_range = C_i_C_n[i][maximum_s_el * s_exponentials + maximum_p_el * p_exponentials : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials + maximum_d_el + d_exponentials]
                j = 0
                d_MO_contributions = 0
                while j <= len(d_coefficient_range):
                    current_contraction_coeffs = d_coefficient_range[j: j + d_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["d"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            d_exp_set = np.append(d_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["d"][0]] * r))
                            d_MO_contributions = d_MO_contributions + current_gaussian_term
                            k += 1
                    j += d_exponentials
                    if j <= len(d_coefficient_range):
                        d_MO_set = np.append(d_MO_set, np.array([d_MO_contributions, i + 1])) 
                        d_ref_set = np.append(d_ref_set, d_MO_contributions)
            else:
                pass
            if len(C_i_C_n.transpose()) > s_exponentials * maximum_s_el + p_exponentials * maximum_p_el + d_exponentials * maximum_d_el:
                f_coefficient_range = C_i_C_n[i][s_exponentials * maximum_s_el + p_exponentials * maximum_p_el + d_exponentials * maximum_d_el : maximum_s_el * s_exponentials + maximum_p_el * p_exponentials + maximum_d_el * d_exponentials + maximum_f_el * f_exponentials]
                j = 0
                f_MO_contributions = 0
                while j <= len(f_coefficient_range):
                    current_contraction_coeffs = d_coefficient_range[j: j + f_exponentials] 
                    k = 0
                    for k in range(0, len(current_contraction_coeffs)):
                        if current_contraction_coeffs[k] != 0:
                            current_gaussian_term = current_contraction_coeffs[k] * sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["f"][0]] * r)
                            gaussian_terms.append(current_gaussian_term)
                            coefficient_sequence.append([current_contraction_coeffs[k]])
                            f_exp_set = np.append(f_exp_set, sympy.exp(-input_tensor[i][exp_range_vals][k + atomic_orbital_states["f"][0]] * r))
                            f_MO_contributions = f_MO_contributions + current_gaussian_term
                            k += 1
                    j += f_exponentials
                    if j <= len(f_coefficient_range):
                        f_MO_set = np.append(f_MO_set, np.array([f_MO_contributions, i])) 
                        f_ref_set = np.append(f_ref_set, f_MO_contributions)
            else:
                pass
            i += 1
        Ci = sympy.symbols("Ci", real = True)
        one_electron_MOs = np.append(one_electron_MOs, s_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, p_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, d_MO_set)
        one_electron_MOs = np.append(one_electron_MOs, f_MO_set)
        exp_set = np.append(exp_set, s_exp_set)
        exp_set = np.append(exp_set, p_exp_set)
        exp_set = np.append(exp_set, d_exp_set)
        exp_set = np.append(exp_set, f_exp_set)
        one_electron_MO_eqns = np.array([])
        k = 0
        for k in range(0, len(s_ref_set) * s_exponentials):
            current_s_MO_eqn = Ci * exp_set[k]
            one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_s_MO_eqn)
            k += 1
        p_gaussian_start_index = (len(s_ref_set) * s_exponentials) - 1
        p_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials) - 1
        p_gaussian_count = p_gaussian_end_index - p_gaussian_start_index
        k = 0
        for k in range(0, int(p_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials])
            current_ref_fxns = list(gaussian_terms[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials * k + p_exponentials]
            current_exponentials = exp_set[p_gaussian_start_index + p_exponentials * k : p_gaussian_start_index + p_exponentials* k + p_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = random.choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
        d_gaussian_start_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials) - 1
        d_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials) - 1
        d_gaussian_count = d_gaussian_end_index - d_gaussian_start_index
        k = 0
        for k in range(0, int(d_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials])
            current_ref_fxns = list(gaussian_terms[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials * k + d_exponentials]
            current_exponentials = exp_set[d_gaussian_start_index + d_exponentials * k : d_gaussian_start_index + d_exponentials* k + d_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = random.choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
        f_gaussian_start_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials) - 1
        f_gaussian_end_index = (len(s_ref_set) * s_exponentials + len(p_ref_set) * p_exponentials + len(d_ref_set) * d_exponentials + len(f_ref_set) * f_exponentials) - 1
        f_gaussian_count = f_gaussian_end_index - f_gaussian_start_index
        k = 0
        for k in range(0, int(f_gaussian_count / 2)):
            current_gaussian_fxns = list(gaussian_terms[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials])
            current_ref_fxns = list(gaussian_terms[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials + 1])
            current_contraction_coefficients = coefficient_sequence[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials * k + f_exponentials]
            current_exponentials = exp_set[f_gaussian_start_index + f_exponentials * k : f_gaussian_start_index + f_exponentials* k + f_exponentials]
            tracker = 1
            prev_gaussian = 0
            for tracker in range(1, len(current_ref_fxns)):
                sample_gaussian = random.choice(current_gaussian_fxns)
                if sample_gaussian != prev_gaussian:
                    gaussian_index = current_gaussian_fxns.index(sample_gaussian)
                    sample_gaussian_eqn = Ci * current_exponentials[gaussian_index]
                    current_gaussian_fxns[gaussian_index] = sample_gaussian_eqn
                    current_gaussian_eqn = sum(current_gaussian_fxns)
                    one_electron_MO_eqns = np.append(one_electron_MO_eqns, current_gaussian_eqn)
                    current_ref_fxns.pop(gaussian_index)
                    prev_gaussian = sample_gaussian
                    current_gaussian_fxns[gaussian_index] = sample_gaussian
                    tracker += 1
                else:
                    continue
                k += 1
                    
        return one_electron_MOs, one_electron_MO_eqns, r

    def compute_effective_density_cutoffs(self, input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, maximum_density_bound):
        single_el_wavefunctions, single_el_wavefunction_eqs, r = self.construct_single_electron_wavefunctions(energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.n_atoms, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el)
        normalized_dist_fxns = np.array([])
        maximum_el_density_radii = np.array([])
        i = 0
        while i < len(single_el_wavefunctions):
            norm_constant = sympy.symbols("A", real = True)
            normalized_wavefunc = norm_constant * single_el_wavefunctions[i]
            total_probability_density = sympy.integrals.integrate(normalized_wavefunc, (r, R_inner, maximum_density_bound))
            norm_constant = sympy.solve(total_probability_density - 1, norm_constant)
            normalized_wavefunc = norm_constant[0] * single_el_wavefunctions[i]
            normalized_dist_fxns = np.append(normalized_dist_fxns, normalized_wavefunc)
            enclosed_probability_density = sympy.integrals.integrate(normalized_wavefunc, r)
            inner_cutoff_single_electron_density = enclosed_probability_density.replace(r, R_inner)
            ith_electron_density_range = ((enclosed_probability_density - inner_cutoff_single_electron_density) - 0.98)**2
            ith_density_range_fxn = sympy.lambdify(r, ith_electron_density_range)
            min_square_enclosed_density = scipy.optimize.minimize_scalar(ith_density_range_fxn)
            ith_maximum_density_radius = min_square_enclosed_density.x
            maximum_el_density_radii = np.append(maximum_el_density_radii, ith_maximum_density_radius)
            maximum_el_density_radii = np.append(maximum_el_density_radii, single_el_wavefunctions[i + 1])
            i += 2
        return maximum_el_density_radii

    
    def MC_functional_integration_estimate(self, input_tensor, n_atoms, atomic_orbital_states, C_i_C_n, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, delta_o_cut, maximum_density_bound, nuclear_separation_matrix, atomic_positions, aromatic_rings, aromatic_cages):  
        single_electron_wavefunctions, single_electron_eqns, r = self.construct_single_electron_wavefunctions(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el)
        r_eff_max = self.compute_effective_density_cutoffs(input_tensor, atomic_orbital_states, C_i_C_n, n_atoms, exponential_range, maximum_s_el, maximum_p_el, maximum_d_el, maximum_f_el, R_inner, maximum_density_bound)
        BCP_bias_decay = -0.1
        # implement either straight-line distance of random var point on sphere from zero-gradient point of single-electron wavefunction, or staight-line distance from random R3 point from R3 line separating nuclei in bond
        MC_estimate = 0
        n_random_samples = 1000
        gradient_eval_samples = 500 #for evaluation of electron density gradient at every pm of the internuclear/ring/cage max effective separation
        sample_count = 0
        prev_gradient_sample = 0
        atomic_numbers = input_tensor.transpose()[2]
        a = 0
        while a < 1:
            max_density_separations = np.array([])
            random_pos_samples = np.array([])
            known_rings = np.array([])
            known_cages = np.array([])
            point_pdf_score = np.array([])
            for j in np.arange(0, len(aromatic_rings)):
                if single_electron_wavefunctions[a+1] in aromatic_rings[j]:
                    nucleus_on_ring = True
                    electron_i_aromatic_ring = j
                    known_rings = np.append(known_rings, j)
                else:
                    nucleus_on_ring = False
            for k in np.arange(0, len(aromatic_cages)):
                if single_electron_wavefunctions[a+1] in aromatic_cages[j]:
                    nucleus_on_cage = True
                    electron_i_cage_surface = j
                    known_cages = np.append(known_cages, j)
                else:
                    nucleus_on_cage = False
            if nucleus_on_ring == True:
                RCPs = np.array([])
                ring_gradient_changes = []
                ring_atom_separations = np.array([])
                ring_density_gradient_step = 0
                for k in np.arange(0, len(aromicatic_rings[electron_i_aromatic_ring])):
                    ring_atom_separations = np.append(ring_atom_separations, np.sqrt((pos_matrix[electron_i_aromatic_ring[j]][0] - pos_matrix[single_electron_wavefunctions[a+1]][0])**2 + (pos_matrix[electron_i_aromatic_ring[j]][1] - pos_matrix[single_electron_wavefunctions[a+1]][1])**2 + (pos_matrix[electron_i_aromatic_ring[j]][2] - pos_matrix[single_electron_wavefunctions[a+1]][2])**2))
                adjusted_max_ring_pos = max(ring_atom_separations) - 2*R_inner
                ring_density_gradient_step = ring_density_gradient_step + (adjusted_max_ring_pos / gradient_eval_samples)
                j = 0
                ring_density_gradient_eval_set = np.array([R_inner])
                while j <= gradient_eval_samples:
                    ring_density_gradient_eval_set = np.append(ring_density_gradient_eval_set, R_inner + j * ring_density_gradient_step)
                    j += 1
                j = 0
                ath_electron_density_surface = single_electron_wavefunctions[a]**2
                ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                while j <= gradient_eval_samples:
                    current_gradient_sample = ath_electron_density_gradient.replace(r, ring_density_gradient_eval_set[j])
                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                    ring_gradient_changes.append(current_gradient_change)
                    prev_gradient_sample = current_gradient_sample
                    j += 1
                gradient_min = min(np.abs(np.array(bond_gradient_changes)))
                gradient_min_separation = ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)]
                preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                next_density_vals = [ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.03)]
                preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                preceeding_max = max(preceeding_density_vals)
                next_max = max(next_density_vals)
                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(ring_atom_separations) - next_max_pos)]) < R_inner):
                    RCPs = np.append(RCPs, gradient_min_separation)
                possible_density_maxima = np.array([R_inner])
                possible_density_maxima = np.append(possible_density_maxima, RCPs)
                for j in np.arange(0, len(ring_atom_separations)):
                    possible_density_maxima = np.append(possible_density_maxima, ring_atom_separations[j] - R_inner)
                least_separation_from_density_max = min(possible_density_maxima)
            elif nucleus_on_cage == True:
                CCPs = np.array([])
                cage_gradient_changes = []
                cage_atom_separations = np.array([])
                cage_density_gradient_step = 0
                for k in np.arange(0, len(aromicatic_cages[electron_i_cage_surface])):
                    cage_atom_separations = np.append(cage_atom_separations, np.sqrt((pos_matrix[electron_i_cage_surface[j]][0] - pos_matrix[single_electron_wavefunctions[a+1]][0])**2 + (pos_matrix[electron_i_cage_surface[j]][1] - pos_matrix[single_electron_wavefunctions[a+1]][1])**2 + (pos_matrix[electron_i_cage_surface[j]][2] - pos_matrix[single_electron_wavefunctions[a+1]][2])**2))
                adjusted_max_cage_pos = max(cage_atom_separations) - 2*R_inner
                cage_density_gradient_step = cage_densiy_gradient_step + (adjusted_max_cage_pos / gradient_eval_samples)
                j = 0
                cage_density_gradient_eval_set = np.array([R_inner])
                while j <= gradient_eval_samples:
                    cage_density_gradient_eval_set = np.append(cage_density_gradient_eval_set, R_inner + j * cage_density_gradient_step)
                    j += 1
                ath_electron_density_surface = single_electron_wavefunctions[a]**2
                ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                j = 0
                while j <= gradient_eval_samples:
                    current_gradient_sample = ath_electron_density_gradient.replace(r, cage_density_gradient_eval_set[j])
                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                    cage_gradient_changes.append(current_gradient_change)
                    prev_gradient_sample = current_gradient_sample
                    j += 1
                gradient_min = min(np.abs(np.array(bond_gradient_changes)))
                gradient_min_separation = cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)]
                preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                next_density_vals = [ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.03)]
                preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                preceeding_max = max(preceeding_density_vals)
                next_max = max(next_density_vals)
                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(cage_atom_separations) - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(cage_atom_separations) - next_max_pos)]) < R_inner):
                    CCPs = np.append(CCPs, gradient_min_separation)
                possible_density_maxima = np.array([R_inner])
                possible_density_maxima = np.append(possible_density_maxima, CCPs)
                for j in np.arange(0, len(cage_atom_separations)):
                    possible_density_maxima = np.append(possible_density_maxima, cage_atom_separations[j] - R_inner)
                least_separation_from_density_max = min(possible_density_maxima)
            else:
                BCPs = np.array([])
                bond_gradient_changes = []
                bond_density_gradient_step = 0
                bond_maximum_separation = max(nuclear_separation_matrix[single_electron_wavefunctions[a+1] - 1])
                adjusted_bond_max_separation = bond_maximum_separation - 2*R_inner
                bond_density_gradient_step = bond_density_gradient_step + (adjusted_bond_max_separation / gradient_eval_samples)
                j = 0
                bond_density_gradient_eval_set = np.array([R_inner])
                while j <= gradient_eval_samples:
                    bond_density_gradient_eval_set = np.append(bond_density_gradient_eval_set, R_inner + j * bond_density_gradient_step)
                    j += 1
                ath_electron_density_surface = single_electron_wavefunctions[a]**2
                ath_electron_density_gradient = sympy.diff(ath_electron_density_surface, r)
                j = 0
                while j <= gradient_eval_samples:
                    current_gradient_sample = ath_electron_density_gradient.replace(r, bond_density_gradient_eval_set[j])
                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                    bond_gradient_changes.append(current_gradient_change)
                    prev_gradient_sample = current_gradient_sample
                    j += 1
                gradient_min = min(np.abs(np.array(bond_gradient_changes)))
                gradient_min_separation = bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)]
                preceeding_density_vals = [ath_electron_density_surface.replace(r, gradient_min_separation - 0.01), ath_electron_density_surface.replace(r, gradient_min_separation - 0.02), ath_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                next_density_vals = [ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.01), ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.02), ath_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.03)]
                preceeding_radial_pos = [gradient_min_separation - 0.01, gradient_min_separation - 0.02, gradient_min_separation - 0.03]
                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                preceeding_max = max(preceeding_density_vals)
                next_max = max(next_density_vals)
                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                if (ath_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(bond_maximum_separation - preceeding_max_pos)]) < R_inner) and (ath_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(bond_maximum_separation - next_max_pos)]) < R_inner):
                    BCPs = np.append(BCPs, gradient_min_separation)
                possible_density_maxima = np.array([R_inner])
                possible_density_maxima = np.append(possible_density_maxima, BCPs)
                for j in np.arange(0, len(nuclear_separation_matrix[single_electron_wavefunctions[a+1]])):
                    if nuclear_separation_matrix[single_electron_wavefunctions[a+1]][j] != 0:
                        possible_density_maxima = np.append(possible_density_maxima, nuclear_separation_matrix[single_electron_wavefunctions[a+1]][j] - R_inner)
                    else:
                        continue
                ath_effective_density_limit = r_eff_max[a]
                print(ath_effective_density_limit)
            while sample_count < n_random_samples:
                random_x = uniform(R_inner, maximum_density_bound)
                random_y = uniform(R_inner, maximum_density_bound)
                random_z = uniform(R_inner, maximum_density_bound)
                if np.sqrt((random_x - pos_matrix[(a+1)][0])**2 + (random_y - pos_matrix[(a+1)][1])**2 + (random_z - pos_matrix[(a+1)][2])**2) <= r_eff_max[a]:
                    random_pos_samples = np.append(random_pos_samples, np.sqrt(random_x**2 + random_y**2 + random_z**2))
                    sample_count += 1
                else:
                    continue
            max_density_separations = np.array([])
            for j in np.arange(0, len(random_pos_samples)):
                for k in np.arange(0, len(possible_density_maxima)):
                    max_density_separations = np.append(max_density_separations, abs(random_pos_samples[j] - possible_density_maxima[k]))
            max_density_separations = max_density_separations.reshape(len(random_pos_samples), len(possible_density_maxima))
            for j in np.arange(0, len(max_density_separations)):
                point_pdf_score = np.append(point_pdf_score, min(max_density_separations[j]))
            for j in np.arange(0, n_atoms):
                v_ext = ath_electron_density_surface * (-atomic_numbers[j] / abs(atomic_positions[j] - r))
                wavefunction_domain_volume= (4/3) * np.pi * ath_effective_density_limit**3
            for k in np.arange(0, n_random_samples):
                BCP_bias = sympy.exp(BCP_bias_decay * point_pdf_score[k])
                PDF = BCP_bias / wavefunction_domain_volume
                KS_functional_sample = v_ext.replace(r, random_pos_samples[k])
                MC_estimate = MC_estimate + KS_functional_sample
            MC_integral_approx = ((1 / PDF) / n_random_samples) + MC_estimate
            a += 2
        return MC_integral_approx
            




j = 0
                for j in range(0, n_atoms):
                    v_min_char_list = []
                    v_max_char_list = []
                    v_min_replacement_str = ""
                    v_max_replacement_str = ""
                    v_ext = ith_electron_density_surface * (-atomic_numbers[j] / abs(atomic_positions[j] - r))
                    v_ext_approx = sympy.series(v_ext)
                    v_ext_i = sympy.integrals.integrate(v_ext_approx, r)
                    v_ext_i_str = str(v_ext_i)
                    k = 0
                    for k in range(0, len(v_ext_i_str)):
                        v_min_char_list.append(v_ext_i_str[k])
                        v_max_char_list.append(v_ext_i_str[k])
                        k += 1
                    k = 0
                    for k in range(0, len(v_min_char_list)):
                        if v_min_char_list[k] == "r":
                            v_min_char_list[k] = str(R_inner)
                        elif v_min_char_list[k] == "O":
                            v_min_drop_index = k
                        else:
                            k += 1
                    del v_min_char_list[v_min_drop_index - 2:]
                    k = 0
                    for k in range(0, len(v_max_char_list)):
                        if v_max_char_list[k] == "r":
                            v_max_char_list[k] = str(max(nuclear_separation_matrix[single_electron_wavefunctions[i+1]]) + delta_o_cut)
                        elif v_max_char_list[k] == "O":
                            v_max_drop_index = k
                        else:
                            k += 1
                    del v_max_char_list[v_max_drop_index - 2:]
                    k = 0
                    for k in range(0, len(v_min_char_list)):
                        v_min_replacement_str =  v_min_replacement_str + v_min_char_list[k]
                    k = 0
                    for k in range(0, len(v_max_char_list)):
                        v_max_replacement_str =  v_max_replacement_str + v_max_char_list[k]
                    v_ext_i_max = sympy.parsing.sympy_parser.parse_expr(v_max_replacement_str)
                    v_ext_i_min = sympy.parsing.sympy_parser.parse_expr(v_min_replacement_str)
                    v_ext_i = v_ext_i_max - v_ext_i_min
                    v_ext_i_placeholder = sympy.symbols("v_ext_i", real = True) 
                    ith_el_external_potential = ith_el_external_potential + v_ext_i
                    ith_el_external_potential = ith_el_external_potential.replace(v_ext_i_placeholder, 0)
                    j += 1
                v_ext_placeholder = sympy.symbols("v_ext", real = True)
                external_potential = external_potential + ith_el_external_potential
                external_potential = external_potential.replace(v_ext_placeholder, 0)
                i += 2


IndentationError: unexpected indent (1858509338.py, line 422)

In [22]:
energy_surface_optimizer = System_Energy_Optimizer(LR_o = 0.2, K = 0.8, t = 0, input_tensor = initial_state_floats, C_i_C_n = H_f_val, H_o_H_f = hidden_states, hidden_state_eqns = hidden_state_eqns, W_o_W_f = weight_eqns, B_o_B_f = bias_eqns, lin_regs = lin_eqns, E_xc = "GGA", n_bonding_electrons = 42, n_atoms = 7, unit_charge = 1.6e-19, atomic_orbital_states = bonding_states, R_inner = 0.01, delta_o_cut = 0.5, maximum_density_bound = 11, gradient_descent_schedule = "MBGD", exponential_range = exponential_range, nuclear_separation_matrix = nuclear_separation_matrix, bond_separation_matrix = bond_separation_matrix, pos_matrix = pos_matrix, degree_vector = degree_vector, atomic_positions = bond_separation_set, maximum_s_el = molecular_graph_details.maximum_s_el, maximum_p_el = molecular_graph_details.maximum_p_el, maximum_d_el = molecular_graph_details.maximum_d_el, maximum_f_el = molecular_graph_details.maximum_f_el, aromatic_rings = aromatic_rings, aromatic_cages = aromatic_cages, orbital_tracker = 0, KS_functional_tracker = "")
#print(energy_surface_optimizer.construct_grad_descent_schedule(energy_surface_optimizer.LR_o, energy_surface_optimizer.t, energy_surface_optimizer.K, energy_surface_optimizer.gradient_descent_schedule))
#basis_set = energy_surface_optimizer.compute_system_energy_eigenval(energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.E_xc, energy_surface_optimizer.n_bonding_electrons, energy_surface_optimizer.n_atoms, energy_surface_optimizer.unit_charge, energy_surface_optimizer.R_inner, energy_surface_optimizer.delta_o_cut, energy_surface_optimizer.maximum_density_bound, energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.exponential_range)
#print(energy_wsurface_optimizer.C_i_C_n[0][energy_surface_optimizer.atomic_orbital_states["p"][1]])
#print(type(energy_surface_optimizer.atomic_orbital_states["p"][1]))
#print(energy_surface_optimizer.exponential_range)
#print(energy_surface_optimizer.compute_system_energy_eigenval(energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.E_xc, energy_surface_optimizer.n_bonding_electrons, energy_surface_optimizer.n_atoms, energy_surface_optimizer.unit_charge, energy_surface_optimizer.R_inner, energy_surface_optimizer.delta_o_cut, energy_surface_optimizer.maximum_density_bound, energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.exponential_range, energy_surface_optimizer.nuclear_separation_matrix, energy_surface_optimizer.bond_separation_matrix, energy_surface_optimizer.degree_vector, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el, energy_surface_optimizer.orbital_tracker, energy_surface_optimizer.KS_functional_tracker, energy_surface_optimizer.pos_matrix))
#maximum_density_radii, domain_overlap_matrix, inner_overlap_distances, outer_overlap_distances = energy_surface_optimizer.compute_effective_density_cutoffs(energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.n_atoms, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el, energy_surface_optimizer.R_inner, energy_surface_optimizer.maximum_density_bound, energy_surface_optimizer.bond_separation_matrix, energy_surface_optimizer.pos_matrix, energy_surface_optimizer.nuclear_separation_matrix)
#print(maximum_density_radii, domain_overlap_matrix, inner_overlap_distances, outer_overlap_distances)
#single_el_orbitals, single_el_eqns, r = energy_surface_optimizer.construct_single_electron_wavefunctions(energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.n_atoms, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el)  
#print(single_el_orbitals)
print(energy_surface_optimizer.MC_functional_integration_estimate(energy_surface_optimizer.input_tensor, energy_surface_optimizer.n_atoms, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el, energy_surface_optimizer.R_inner, energy_surface_optimizer.delta_o_cut, energy_surface_optimizer.maximum_density_bound, energy_surface_optimizer.nuclear_separation_matrix, energy_surface_optimizer.atomic_positions, energy_surface_optimizer.aromatic_rings, energy_surface_optimizer.aromatic_cages, energy_surface_optimizer.orbital_tracker, energy_surface_optimizer.KS_functional_tracker))
#print(energy_surface_optimizer.test_bond_max_algorithm(energy_surface_optimizer.nuclear_separation_matrix, energy_surface_optimizer.degree_vector, energy_surface_optimizer.delta_o_cut, energy_surface_optimizer.input_tensor, energy_surface_optimizer.atomic_orbital_states, energy_surface_optimizer.C_i_C_n, energy_surface_optimizer.n_atoms, energy_surface_optimizer.exponential_range, energy_surface_optimizer.maximum_s_el, energy_surface_optimizer.maximum_p_el, energy_surface_optimizer.maximum_d_el, energy_surface_optimizer.maximum_f_el))                                                  

(7, 7)


In [ ]:
 while i < 84:
            print(i)
            for j in np.arange(0, len(aromatic_rings)):
                if single_electron_wavefunctions[i+1] in aromatic_rings[j]:
                    nucleus_on_ring = True
                    electron_i_aromatic_ring = j
                    known_rings = np.append(known_rings, j)
                else:
                    nucleus_on_ring = False
            for k in np.arange(0, len(aromatic_cages)):
                if single_electron_wavefunctions[i+1] in aromatic_cages[j]:
                    nucleus_on_cage = True
                    electron_i_cage_surface = j
                    known_cages = np.append(known_cages, j)
                else:
                    nucleus_on_cage = False
            if nucleus_on_ring == True:
                ring_atom_separations = np.array([])
                for k in np.arange(0, len(aromicatic_rings[electron_i_aromatic_ring])):
                    ring_atom_separations = np.append(ring_atom_separations, np.sqrt(pos_matrix[single_electron_wavefunctions[i+1]][0]**2 + pos_matrix[single_electron_wavefunctions[i+1]][1]**2 + pos_matrix[single_electron_wavefunctions[i+1]][2]**2))
                ring_density_gradient_steps = np.append(ring_density_gradient_steps, (max(ring_atom_separations) - 2*R_inner) / gradient_eval_samples)
                j = 0
                while j <= gradient_eval_samples:
                    ring_density_gradient_eval_set = np.append(ring_density_gradient_eval_set, R_inner + j * density_gradient_steps[i])
                    j += 1
                j = 0
                while j <= gradient_eval_samples:
                    current_gradient_sample = ith_electron_density_gradient.replace(r, ring_density_gradient_eval_set[j])
                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                    ring_gradient_changes.append(current_gradient_change)
                    prev_gradient_sample = current_gradient_sample
                    j += 1
                gradient_min = min(ring_gradient_changes)
                gradient_min_separation = ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)]
                preceeding_density_vals = [ith_electron_density_surface.replace(r, gradient_min_separation - 0.01), ith_electron_density_surface.replace(r, gradient_min_separation - 0.02), ith_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                next_density_vals = [ith_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.01), ith_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.02), ith_electron_density_surface.replace(r, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] + 0.03)]
                preceeding_radial_pos = [ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] - 0.01, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] - 0.02, ring_density_gradient_eval_set[ring_gradient_changes.index(gradient_min)] - 0.03]
                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                preceeding_max = max(preceeding_density_vals)
                next_max = max(next_density_vals)
                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                if (ith_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(ring_atom_separations) - preceeding_max_pos)]) < R_inner) and (ith_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(ring_atom_separations) - next_max_pos)]) < R_inner):
                    RCPs = np.append(RCPs, gradient_min_separation)
                possible_density_maxima = np.append(possible_density_maxima, R_inner)
                possible_density_maxima = np.append(possible_density_maxima, RCPs)
                for j in np.arange(0, len(ring_atom_separations)):
                     possible_density_maxima = np.append(possible_density_maxima, ring_atom_separations[j] - R_inner)
                least_separation_from_density_max = min(possible_density_maxima)
            elif nucleus_on_cage == True:
                for k in np.arange(0, len(aromicatic_cages[electron_i_cage_surface])):
                    cage_atom_separations = np.append(cage_atom_separations, np.sqrt(pos_matrix[single_electron_wavefunctions[i+1]][0]**2 + pos_matrix[single_electron_wavefunctions[i+1]][1]**2 + pos_matrix[single_electron_wavefunctions[i+1]][2]**2))
                cage_density_gradient_steps = np.append(cage_density_gradient_steps, (max(cage_atom_separations) - 2*R_inner) / gradient_eval_samples)
                j = 0
                while j <= gradient_eval_samples:
                    cage_density_gradient_eval_set = np.append(cage_density_gradient_eval_set, R_inner + j * density_gradient_steps[i])
                    j += 1
                j = 0
                while j <= gradient_eval_samples:
                    current_gradient_sample = ith_electron_density_gradient.replace(r, cage_density_gradient_eval_set[j])
                    current_gradient_change = current_gradient_sample - prev_gradient_sample
                    cage_gradient_changes.append(current_gradient_change)
                    prev_gradient_sample = current_gradient_sample
                    j += 1
                gradient_min = min(bond_gradient_changes)
                gradient_min_separation = cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)]
                preceeding_density_vals = [ith_electron_density_surface.replace(r, gradient_min_separation - 0.01), ith_electron_density_surface.replace(r, gradient_min_separation - 0.02), ith_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
                next_density_vals = [ith_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.01), ith_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.02), ith_electron_density_surface.replace(r, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] + 0.03)]
                preceeding_radial_pos = [cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] - 0.01, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] - 0.02, cage_density_gradient_eval_set[cage_gradient_changes.index(gradient_min)] - 0.03]
                next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
                preceeding_max = max(preceeding_density_vals)
                next_max = max(next_density_vals)
                preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
                next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
                if (ith_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(max(cage_atom_separations) - preceeding_max_pos)]) < R_inner) and (ith_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(max(cage_atom_separations) - next_max_pos)]) < R_inner):
                    CCPs = np.append(CCPs, gradient_min_separation)
                possible_density_maxima = cage_atom_separations
                possible_density_maxima = np.append(possible_density_maxima, CCPs)
                least_separation_from_density_max = min(possible_density_maxima)
            else:
                bond_maximum_separation = max(nuclear_separation_matrix[single_electron_wavefunctions[i+1]])
                bond_density_gradient_steps = np.append(bond_density_gradient_steps, bond_maximum_separation / gradient_eval_samples)
            i += 2
            #return single_electron_wavefunctions
               # j = 0
               # while j <= gradient_eval_samples:
                    #bond_density_gradient_eval_set = np.append(bond_density_gradient_eval_set, R_inner + j * bond_density_gradient_steps[i])
                    #j += 1
            #ith_electron_density_surface = single_electron_wavefunctions[i+1]**2
            #ith_electron_density_gradient = sympy.diff(ith_electron_density_surface, r)
            #j = 0
            #while j <= gradient_eval_samples:
                #current_gradient_sample = ith_electron_density_gradient.replace(r, bond_density_gradient_eval_set[j])
                #current_gradient_change = current_gradient_sample - prev_gradient_sample
                #bond_gradient_changes.append(current_gradient_change)
                #prev_gradient_sample = current_gradient_sample
                #j += 1
            #gradient_min = min(bond_gradient_changes)
            #gradient_min_separation = bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)]
            #preceeding_density_vals = [ith_electron_density_surface.replace(r, gradient_min_separation - 0.01), ith_electron_density_surface.replace(r, gradient_min_separation - 0.02), ith_electron_density_surface.replace(r, gradient_min_separation - 0.03)]
            #next_density_vals = [ith_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.01), ith_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.02), ith_electron_density_surface.replace(r, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] + 0.03)]
            #preceeding_radial_pos = [bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] - 0.01, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] - 0.02, bond_density_gradient_eval_set[bond_gradient_changes.index(gradient_min)] - 0.03]
            #next_radial_pos = [gradient_min_separation + 0.01, gradient_min_separation + 0.02, gradient_min_separation + 0.03]
            #preceeding_max = max(preceeding_density_vals)
            #next_max = max(next_density_vals)
            #preceeding_max_pos = preceeding_radial_pos[preceeding_density_vals.index(preceeding_max)]
            #next_max_pos = next_radial_pos[next_density_vals.index(next_max)]
            #if (ith_electron_density_surface.replace(r, gradient_min_separation) > preceeding_max or min([preceeding_max_pos, abs(bond_maximum_separation - preceeding_max_pos)]) < R_inner) and (ith_electron_density_surface.replace(r, gradient_min_separation) > next_max or min([next_max_pos, abs(bond_maximum_separation - next_max_pos)]) < R_inner):
                #BCPs = np.append(BCPs, gradient_min_separation)
            #possible_density_maxima = cage_atom_separations
            #possible_density_maxima = np.append(possible_density_maxima, BCPs)
       # while sample_count < n_random_samples:
            #random_x = randrange(R_inner, maximum_density_bound)
            #random_y = randrange(R_inner, maximum_density_bound)
            #random_z = randrange(R_inner, maximum_density_bound)
            #if np.sqrt((random_x - pos_matrix[i][0])**2 + (random_y - pos_matrix[i][1])**2 + (random_z - pos_matrix[i][2])**2) <= r_eff_max[i]:
                #random_pos_samples = np.append(random_pos_samples, np.sqrt(random_x**2 + random_y**2 + random_z**2))
                #sample_count += 1
            #else:
                #continue
        #possible_density_maxima = np.arrray([])
        #max_density_separations = np.array([])
        #for j in np.arange(0, len(random_pos_samples)):
            #for k in np.arange(0, len(possible_density_maxima)):
                #max_density_separations = np.append(max_density_separations, abs(random_pos_samples[j] - possible_density_separations[k]))
        #max_density_separations = max_density_separations.reshape(len(random_pos_samples), len(possible_density_maxima))
       # for j in np.arange(0, len(max_density_separations)):
            #point_pdf_score = np.append(point_pdf_score, min(max_density_separations[j]))
        #for j in np.arange(0, n_atoms):
            #v_ext = ith_electron_density_surface * (-atomic_numbers[j] / abs(atomic_positions[j] - r))
           # wavefunction_domain_volume= (4/3) * np.pi * r_eff_max[i]**3
            #for k in np.arange(0, n_random_samples):
                #BCP_bias = sympy.exp(BCP_bias_decay * point_pdf_score[k])
                #PDF = BCP_bias / wavefunction_domain_volume
                #KS_functional_sample = v_ext.replace(r, random_pos_samples[k])
                #MC_estimate = MC_estimate + KS_functional_sample
            #MC_integral_approx = ((1 / PDF) / n_random_samples) + MC_estimate
        #i += 2
        #return MC_integral_approx


In [20]:
print(pos_matrix)

[[ 0.    0.    0.  ]
 [-1.2  -2.19  0.  ]
 [-2.66  1.6   0.  ]
 [ 0.    5.4   0.  ]
 [ 1.62  1.    0.  ]
 [ 1.9  -1.92  0.  ]
 [ 0.   -4.7   0.  ]]


In [25]:
# use 1-2 varibale MC integration to compute eigenvalues of KS potential energy functionals using random var sampling with bias along bond paths for all r or (r, r_prime) points within the maximum density radius of the nucleus from which the single-electron orbital in question originates: (also add to energy optimizer class)
# remember: if 2 electrons both participating in the same bound are used in calculation of coulomb potential, compute overlap integral between M.O's of the 2 different electrons, then evaluate MC integration for the spherical sets of radial points between R_inner and maximum bound with 0 overlap in probability density for each electron separately, then add result of MC integration for coulomb functional for spherical set of points with probability density overlap.
# paper links: https://deepblue.lib.umich.edu/bitstream/handle/2027.42/70465/JCPSA6-56-9-4419-1.pdf;sequence=2
# https://pbr-book.org/4ed/Monte_Carlo_Integration/Monte_Carlo_Basics

    

  


KS_functional = 
#print(single_el_wavefunctions)
#r = sympy.symbols("r", real = True)
#r_prime = sympy.symbols("r_prime", real = True)
#pos_var_list = [r, r_prime]
#ex_fxn = 2.5e+10 * sympy.exp(0.53 * r) + 3.3e+10 * sympy.exp(2.2 * r) + (1/(sympy.exp(2.72 * r_prime)))
#new_ex_fxn = ex_fxn * (235/(0.009-r))
#reference_val = [0.01, 0.01]
#degree = 5
#print(energy_component_series_approx(new_ex_fxn, pos_var_list, reference_val, degree))
#print(energy_component_series_approx(new_ex_fxn, pos_var_list, reference_val, degree).replace(r, reference_val[0] + 2).replace(r_prime, reference_val[1] + 2))
#print(new_ex_fxn.replace(r, reference_val[0] + 2).replace(r_prime, reference_val[1] + 2))

#test_series = sympy.series(new_ex_fxn, r_prime)
#test_series = sympy.series(test_series, r)
#print(test_series)
#simplified_fxn = 2.5e+10 * sympy.exp(0.53 * r) + 3.3e+10 * sympy.exp(2.2 * r)
#print(energy_component_series_approx(simplified_fxn, [r], [0.01], 5))
#series = sympy.series(new_ex_fxn, r2, x0 = 0.01)
#series = sympy.
#print(series)
#series_int = sympy.integrals.integrate(series, r)
#series_int_str = "log(2.997) + " + str(series_int) 
#series_int_str_replacement = ""
#series_char_list = []
#for i in range(0, len(series_int_str)):
   #series_char_list.append(series_int_str[i])
#for j in range(0, len(series_char_list)):
    #if series_char_list[j] == "r":
        #series_char_list[j] = str(10)
    #elif series_char_list[j] == "O":
        #drop_index = j
    #else:
        #j += 1
#del series_char_list[drop_index - 2:]
#for k in range(0, len(series_char_list)):
    #series_int_str_replacement = series_int_str_replacement + series_char_list[k]
#series_int = sympy.parsing.sympy_parser.parse_expr(series_int_str_replacement)


[4.8384513726034*exp(-4.62*r) 4.8384513726034*exp(-4.62*r)
 12.254282112114*exp(-10.98*r) 12.254282112114*exp(-10.98*r)
 4.03350271109592*exp(-3.88*r) 4.03350271109592*exp(-3.88*r)
 2.93239548478395*exp(-2.85*r) 2.93239548478395*exp(-2.85*r)
 2.62638950400997*exp(-2.56*r) 2.62638950400997*exp(-2.56*r)
 3.2934018875625*exp(-3.19*r) 3.2934018875625*exp(-3.19*r)
 4.95910461147127*exp(-4.73*r) 4.95910461147127*exp(-4.73*r)
 0.311313148682589*exp(-2.91*r) + 0.311313148682589*exp(-0.244*r)
 0.31131314868259*exp(-2.91*r) + 0.31131314868259*exp(-0.244*r)
 0.31131314868259*exp(-2.91*r) + 0.31131314868259*exp(-0.244*r)
 0.31131314868259*exp(-2.91*r) + 0.31131314868259*exp(-0.244*r)
 1.52695675706325*exp(-3.15*r) + 1.52695675706325*exp(-2.8*r)
 1.52695675706325*exp(-3.15*r) + 1.52695675706325*exp(-2.8*r)
 1.52695675706325*exp(-3.15*r) + 1.52695675706325*exp(-2.8*r)
 1.52695675706324*exp(-3.15*r) + 1.52695675706324*exp(-2.8*r)
 0.69532621842468*exp(-2.98*r) + 0.69532621842468*exp(-0.88*r)
 0.69532

In [ ]:
ex_seq = np.arange(0, 5, 10e-5)
rand = np.random.choice(np.random.permutation(ex_seq))
print(np.sqrt(rand))

In [29]:
i = 0
while i < energy_surface_optimizer.n_bonding_electrons:
    print(pos_matrix[single_el_orbitals[2*i + 1] - 1])
    print(single_el_orbitals[2*i+1])
    i += 1
print(pos_matrix)

[0. 0. 0.]
1
[0. 0. 0.]
1
[-1.2  -2.19  0.  ]
2
[-1.2  -2.19  0.  ]
2
[-2.66  1.6   0.  ]
3
[-2.66  1.6   0.  ]
3
[0.  5.4 0. ]
4
[0.  5.4 0. ]
4
[1.62 1.   0.  ]
5
[1.62 1.   0.  ]
5
[ 1.9  -1.92  0.  ]
6
[ 1.9  -1.92  0.  ]
6
[ 0.  -4.7  0. ]
7
[ 0.  -4.7  0. ]
7
[ 0.  -4.7  0. ]
0
[ 0.  -4.7  0. ]
0
[ 0.  -4.7  0. ]
0
[ 0.  -4.7  0. ]
0
[0. 0. 0.]
1
[0. 0. 0.]
1
[0. 0. 0.]
1
[0. 0. 0.]
1
[-1.2  -2.19  0.  ]
2
[-1.2  -2.19  0.  ]
2
[-1.2  -2.19  0.  ]
2
[-1.2  -2.19  0.  ]
2
[-2.66  1.6   0.  ]
3
[-2.66  1.6   0.  ]
3
[-2.66  1.6   0.  ]
3
[-2.66  1.6   0.  ]
3
[0.  5.4 0. ]
4
[0.  5.4 0. ]
4
[0.  5.4 0. ]
4
[0.  5.4 0. ]
4
[1.62 1.   0.  ]
5
[1.62 1.   0.  ]
5
[1.62 1.   0.  ]
5
[1.62 1.   0.  ]
5
[ 1.9  -1.92  0.  ]
6
[ 1.9  -1.92  0.  ]
6
[ 1.9  -1.92  0.  ]
6
[ 1.9  -1.92  0.  ]
6
[[ 0.    0.    0.  ]
 [-1.2  -2.19  0.  ]
 [-2.66  1.6   0.  ]
 [ 0.    5.4   0.  ]
 [ 1.62  1.    0.  ]
 [ 1.9  -1.92  0.  ]
 [ 0.   -4.7   0.  ]]


In [19]:
 if prev_flag == True:
                        xrange = np.arange(-directional_density_limit[0], directional_density_limit[0], 10e-3)
                        yrange = np.arange(-directional_density_limit[0], directional_density_limit[0], 10e-3)
                        zrange = np.arange(-directional_density_limit[0], directional_density_limit[0], 10e-3)
                        random_x = np.random.choice(np.random.permutation(xrange))
                        random_y = np.random.choice(np.random.permutation(yrange))
                        random_z = np.random.choice(np.random.permutation(zrange))
                        for j in np.arange(0, len(nucleus_a_inner_cutoffs)):
                            if nucleus_a_inner_cutoffs[j] < (random_x**2 + random_y**2 + random_z**2)**(1/2) < corrected_nuclear_separations[j]:
                                inner_cutoff_flag = True
                        print((random_x**2 + random_y**2 + random_z**2)**(1/2))
                        print(inner_cutoff_flag, sample_count)
                        if inner_cutoff_flag == True:
                            sample_count += 1
                            prev_flag = True
                        elif (random_x**2 + random_y**2 + random_z**2)**(1/2) >= R_inner and (random_x**2 + random_y**2 + random_z**2)**(1/2) <= r_eff_max[a]:
                            random_pos_samples = np.append(random_pos_samples, (random_x**2 + random_y**2 + random_z**2)**(1/2))  
                            prev_flag = False
                            sample_count += 1
                        else:
                            continue 

2.0


In [29]:
print(energy_surface_optimizer.n_bonding_electrons * 2)

84
